#  AUTHORITATIVE VERSION AS OF 01/25

## Stuff for Zea

In [ ]:
## Import libraries
import numpy as np
import scipy.integrate as integrate
import time
import csv
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

## Ignore dumb warnings. We are adults. We take our chances
import warnings
warnings.filterwarnings('ignore')

# Make plots look good according to KY
import matplotlib.style as style
plt.rcParams.update(plt.rcParamsDefault)
style.use('seaborn-v0_8-poster')
plt.rcParams['figure.dpi'] = 300

plt.rcParams.update(plt.rcParamsDefault)

'''
Fonts
'''
plt_font_size = 36
lgd_font_size = 24
plt.rcParams['font.family'] = "TeX Gyre Termes"
#plt.rc('font', **{'family' : 'sans-serif', 'sans-serif' : ['Myriad Pro']})
plt.rcParams['font.size'] = plt_font_size
plt.rcParams['axes.labelsize'] = plt_font_size
plt.rcParams['axes.titlesize'] = plt_font_size
plt.rcParams['xtick.labelsize'] = plt_font_size
plt.rcParams['ytick.labelsize'] = plt_font_size
plt.rcParams['legend.fontsize'] = lgd_font_size

'''
Lines
'''
plt.rcParams['lines.linewidth'] = 3
plt.rcParams['lines.markeredgewidth'] = 3
plt.rcParams['errorbar.capsize'] = 5
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['xtick.major.size'] = 10
plt.rcParams['xtick.major.width'] = 3
plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['xtick.minor.width'] = 3
plt.rcParams['ytick.major.size'] = 10
plt.rcParams['ytick.major.width'] = 3
plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['ytick.minor.width'] = 3
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

'''
LaTeX
'''
plt.rcParams['text.usetex'] = True
plt.rcParams['text.latex.preamble'] = '\n'.join([
    r'\usepackage[T3,T1]{fontenc}',
    r'\DeclareSymbolFont{tipa}{T3}{cmr}{m}{n}',
    r'\DeclareMathAccent{\invbreve}{\mathalpha}{tipa}{16}',
    r'\usepackage{siunitx}'
])  # Preamble must be one line!

data_path = "./zea-data/"

In [ ]:
from DU4 import *

In [ ]:
def mec_model(t, y, crop, CO2, PPF):
    '''
    Differential equation model. 
    :param t: time in days after emergence
    :type t: float
    :param y: model variable
    :type y: model variable
    :param crop: crop object
    :type crop: object
    :param CO2: atmospheric concentration of carbon dioxide in [umol_CO2/mol_air]
    :type CO2: float
    :param PPF: photosynthetic photon flux in [umol_photon / (m^2 * s)]
    :type PPF: float
    :return: dydt
    :rtype: array
    '''

    ## Set state variables
    # B = y[0]
    # O2 = y[1]
    # H2O = y[2]
    Neq = len(y)

    ## Prepare dydt array
    dydt = np.zeros((1, Neq))

    ## Define dydt
    dydt[0, 0] = calc_m_B(t, crop, CO2, PPF)
    dydt[0, 1] = calc_n_O2(t, crop, CO2, PPF)
    # dydt[0,2] = calc_transpiration(t,crop,CO2,PPF)
    dydt[0, 2] = calc_V_trs(t, crop, CO2, PPF) - calc_H2O_in(t, crop, CO2, PPF)
    # dydt[0,2] = -calc_H2O_in(t,crop,CO2,PPF) * crop.H2O_uptake

    # dydt[0,1] = calc_DOP(t,CO2,PPF,crop)

    return [np.transpose(dydt)]

In [ ]:
## Define directory and locations
directory = 'parameter-lists/'
filename = 'crop_parameters_FPSD.xlsx'

## Load standard parameters from BVAD
filename_full = directory + filename

## Load the standard lettuce crop
crop_type = 'lettuce'
crop = Crop(crop_type, filename_full=filename_full)

#override harvest time
crop.t_M = 50

tspan =  [0, crop.t_M]
y0 = [0,0,50]
# Normal lettuce params are CO2=800, PPF=350. Lettuce PPF is [200,500].
# model's ranges are CO2=[330,1300] and PPF=[200,1000]. Rice and wheat can do PPF=2000.
CO2 = 525
PPF = 225

## Perform integration
start_time = time.time()
sol_MEC = integrate.solve_ivp(mec_model, tspan, y0, args=(crop,CO2,PPF), method='LSODA', max_step=1)
print("--- %s seconds ---" % (time.time() - start_time))

## Convert m_T to m_E (with lettuce t_E = 0)
# sol_MEC.y[0] *= crop.f_E

# single plot just for this
# fig, ax1 = plt.subplots(1, figsize=(10, 10))
# plt.plot(sol_MEC.t, sol_MEC.y[0],linewidth=4, color = 'Lime', label=r'Biomass [g/m$^{2}$]')
# # plt.plot(sol_MEC.t, sol_MEC.y[1],linewidth=4, color = 'b', label=r'O$_{2}$ [g/m$^{2}$]')
# # plt.plot(sol_MEC.t, sol_MEC.y[2],linewidth=4, color = 'k', label=r'H$_{2}$O [g/m$^{2}$]')
# plt.ylabel('Edible Biomass [\si{\gram\of{DW}\per\meter\squared}]')
# plt.xlabel('Time, t [d$_{\mathrm{AE}}$]')
# ax1.set_title('ODE MEC Model of Biomass Growth')
# # plt.legend()
# plt.tight_layout()
# plt.show()

## Plant measurements

In [ ]:
class df_selection:
    """
    This class filters by N setting and then averages mass and finds SD
    
    Use when working only with mB
    """
    def __init__(self, df, var, setting):
        self.statted = df[df[var] == setting].groupby(time_label, as_index=False)[mass_label].agg([np.mean, np.std])
        
class df_n_selection:
    """
    This class only filters a DF by N setting
    """
    def __init__(self, df, var, setting):
        self.grouped = df[df[var] == setting]

global mass_label
mass_label = "mass_B"
global n_mass_label
n_mass_label = "mass_N"
global nitrogen_label 
nitrogen_label = "N_setting"
global time_label 
time_label = "day"

sigma_N_zea = 37.0278518 #Numerical density, from Zea, [3.44 ft_^-2 -> m^-2]
sigma_N_nasa = 19.2 #from BVAD, [N/m^-2]
sigma_N = sigma_N_nasa
water_cont = 0.95 #from BVAD & experiment
harvest_cutoff = 41 #for MEC; days AE
point_offset = 0.2 #avoid overlapping data points
point_offset_mec = 0.5 #avoid overlapping data points
'''>>>>> AVG. EMERGENCE 2021/07/01, 3 DAYS AFTER SOWING <<<<<'''
emerge_offset = -3 #Zea counts from planting day; we want emergence.
harvest_cutoff_zea = harvest_cutoff - emerge_offset

# hg_mB_color="DarkGreen"
# md_mB_color="Lime"
# lw_mB_color="#7fb600"

hg_color="MidnightBlue"
md_color="DodgerBlue"
lw_color="SkyBlue"

hg_label="Excess N Condition (12.5 mM)"
md_label="Normal N Condition (7.5 mM)"
lw_label="Deficient N Condition (2.5 mM)"

time_xlabel = "Time, $t\ [\si{\day\of{AE}}]$"

def_ln_wt = 5

## Plot biomass over time

In [ ]:
'''
Figure: mB

We still calculate this one rather than reading from a file.
df_selection returns means and SDs

For now, let's only show this vs. MEC
'''
hg_mB_color=hg_color
md_mB_color=md_color
lw_mB_color=lw_color

mB_lnwt = def_ln_wt
mB_ls = '-'
mB_plt_param={'ls':mB_ls, 'lw':mB_lnwt}

#read in the raw data
zea_mB = pd.read_csv(data_path + 'zea_masses.csv')
zea_mB = zea_mB[zea_mB[time_label] <= harvest_cutoff_zea]

zea_mB_hg = df_selection(zea_mB, nitrogen_label, 1)
zea_mB_md = df_selection(zea_mB, nitrogen_label, 0)
zea_mB_lw = df_selection(zea_mB, nitrogen_label, -1)

# had to add this block because pandas started acting differently
zea_mB_hg.statted = zea_mB_hg.statted.set_index([time_label])
zea_mB_md.statted = zea_mB_md.statted.set_index([time_label])
zea_mB_lw.statted = zea_mB_lw.statted.set_index([time_label])

zea_mB_hg.statted.index = zea_mB_hg.statted.index + point_offset_mec + emerge_offset
zea_mB_md.statted.index += emerge_offset
zea_mB_lw.statted.index = zea_mB_lw.statted.index - point_offset_mec + emerge_offset


'''
ax1=zea_mB_hg.statted.plot(y="mean", yerr="std", label=hg_label, color=hg_mB_color)
zea_mB_md.statted.plot(y="mean", yerr="std", label=md_label, ax=ax1, color=md_mB_color)
zea_mB_lw.statted.plot(y="mean", yerr="std", label=lw_label, ax=ax1, color=lw_mB_color)
ax1.set_xlabel(time_xlabel)
ax1.set_ylabel("Fresh weight biomass, $m_\mathrm{B}\ [\si{\kilo\gram\of{FW}}]$")
plt.show()
'''

'''
Figure: mB->dwa data vs. MEC as per area DW. dwa = dry-weight-area
'''
mec_df = pd.DataFrame({'t': sol_MEC.t, 'mB': sol_MEC.y[0]})
mec_df = mec_df[(mec_df['t'] <= harvest_cutoff) & (mec_df['t'] >= 1)]

zea_mB_dwa_hg = zea_mB_hg.statted * sigma_N * (1 - water_cont)
zea_mB_dwa_md = zea_mB_md.statted * sigma_N * (1 - water_cont)
zea_mB_dwa_lw = zea_mB_lw.statted * sigma_N * (1 - water_cont)

# Figure
fig_mB, ax_mB = plt.subplots(1, figsize=(10, 10))
ax_mB.set_box_aspect(1)

# Draw plots
zea_mB_dwa_hg.plot(y = "mean", yerr = "std", label=hg_label, color=hg_mB_color, **mB_plt_param, ax=ax_mB)
zea_mB_dwa_md.plot(y = "mean", yerr = "std", label=md_label, color=md_mB_color, **mB_plt_param, ax=ax_mB)
zea_mB_dwa_lw.plot(y = "mean", yerr = "std", label=lw_label, color=lw_mB_color, **mB_plt_param, ax=ax_mB)
mec_df.plot(x = 't', y = "mB", label = "Modified Energy Cascade", color="LimeGreen", style='--', ax=ax_mB, lw=mB_lnwt)

# Set axes
ax_mB.set_xlabel(time_xlabel)
# ax_mB.set_xlim(0, harvest_cutoff + 1)
# plt.xticks(np.arange(0, harvest_cutoff + 1, 5))
ax_mB.set_xlim(0.5,40.9)
ax_mB.xaxis.set_major_locator(ticker.MultipleLocator(5))
ax_mB.xaxis.set_minor_locator(ticker.MultipleLocator(1))

ax_mB.set_ylabel("Areal dry weight biomass, $\invbreve{m}_\mathrm{B}\ [\si{\gram\of{DW}\per\meter\squared}]$")
ax_mB.yaxis.set_minor_locator(ticker.MultipleLocator(10))

# Draw t_M box and label
ax_mB.axvspan(30, 35, color="#a5a5a5", alpha=0.25) # label="Harvesting Range"
ax_mB.text(32.5, ax_mB.get_ylim()[1]*0.9, r"$t_\textrm{M}$", fontsize=lgd_font_size+2, ha='center')

# Legend
# handles, labels = plt.gca().get_legend_handles_labels()
# #order=[2,3,1,0]
# order=[1,2,3,0]
# plt.legend([handles[i] for i in order],[labels[i] for i in order], ncol=2, loc="lower center",  bbox_to_anchor=(0.5, -0.35))
ax_mB.get_legend().remove()

# Draw legend separately
handles, labels = plt.gca().get_legend_handles_labels()
order=[1,2,3,0]
fig_mB_l, ax_mB_l = plt.subplots()
ax_mB_l.axis(False)
ax_mB_l.legend([handles[i] for i in order],[labels[i] for i in order], loc="center", ncol=2)

plt.show()

fig_mB.savefig("./NP-images/MEC-vs-N.png", dpi=300, bbox_inches='tight', transparent=True)
fig_mB_l.savefig("./NP-images/MEC-legend.png", dpi=300, bbox_inches='tight', transparent=True)

## Plot change in biomass over time

In [ ]:
'''
Figure: dmB/dt

We used to calculate this, but because of SDs, we read from a file.
'''
#zea_df_m_hg_d = zea_df_m_hg.statted["mean"].diff() / zea_df_m_hg.statted.index.to_series().diff()
#zea_df_m_md_d = zea_df_m_md.statted["mean"].diff() / zea_df_m_md.statted.index.to_series().diff()
#zea_df_m_lw_d = zea_df_m_lw.statted["mean"].diff() / zea_df_m_lw.statted.index.to_series().diff()

#ax_dmBdt = zea_df_m_hg_d.plot(label=hg_label, color=hg_color, style='.--')
#zea_df_m_md_d.plot(label=md_label, color=md_color, style='.--', ax=ax_dmBdt)
#zea_df_m_lw_d.plot(label=lw_label, color=lw_color, style='.--', ax=ax_dmBdt)

dmBdt_lnwt = def_ln_wt
dmBdt_ls = '--'
dmbdt_plt_param={'ls':dmBdt_ls, 'lw':dmBdt_lnwt}

zea_dmbdt = pd.read_csv(data_path + 'zea_dmbdt.csv')
zea_dmbdt = zea_dmbdt[zea_dmbdt[time_label] <= harvest_cutoff_zea]
zea_dmbdt_hg = zea_dmbdt[zea_dmbdt[nitrogen_label] == 1]
zea_dmbdt_md = zea_dmbdt[zea_dmbdt[nitrogen_label] == 0]
zea_dmbdt_lw = zea_dmbdt[zea_dmbdt[nitrogen_label] == -1]

zea_dmbdt_hg[time_label] = zea_dmbdt_hg[time_label] + point_offset + emerge_offset
zea_dmbdt_md[time_label] += emerge_offset
zea_dmbdt_lw[time_label] = zea_dmbdt_lw[time_label] - point_offset + emerge_offset

# This keeps all shapes the same but changes the y scale for FW > DW
zea_dmbdt_hg["dmb/dt"] *= (1 - water_cont)
zea_dmbdt_hg["SD"] *= (1 - water_cont)
zea_dmbdt_md["dmb/dt"] *= (1 - water_cont)
zea_dmbdt_md["SD"] *= (1 - water_cont)
zea_dmbdt_lw["dmb/dt"] *= (1 - water_cont)
zea_dmbdt_lw["SD"] *= (1 - water_cont)

# Figure
fig_dmBdt, ax_dmBdt = plt.subplots(1, figsize=(10, 10))
ax_dmBdt.set_box_aspect(1)

# Draw plots
zea_dmbdt_hg.plot(x=time_label, y="dmb/dt", yerr="SD", label=hg_label, color=hg_mB_color, **dmbdt_plt_param, ax=ax_dmBdt)
zea_dmbdt_md.plot(x=time_label, y="dmb/dt", yerr="SD", label=md_label, color=md_mB_color, **dmbdt_plt_param, ax=ax_dmBdt)
zea_dmbdt_lw.plot(x=time_label, y="dmb/dt", yerr="SD", label=lw_label, color=lw_mB_color, **dmbdt_plt_param, ax=ax_dmBdt)

# Set axes
ax_dmBdt.set_xlabel(time_xlabel)
plt.xticks(np.arange(20, harvest_cutoff + 1, 5))
ax_dmBdt.set_xlim(19.5,40.5)
ax_dmBdt.xaxis.set_major_locator(ticker.MultipleLocator(5))
ax_dmBdt.xaxis.set_minor_locator(ticker.MultipleLocator(1))

ax_dmBdt.set_ylabel("Biomass Growth Rate, $\dot{m}_\mathrm{B}\ [\si{\gram\of{DW}\per\day}]$")
ax_dmBdt.yaxis.set_minor_locator(ticker.MultipleLocator(2.5))

# Draw t_M box and label
ax_dmBdt.axvspan(30, 35, color="#a5a5a5", alpha=0.25) # label="Harvesting Range"
ax_dmBdt.text(32.5, ax_dmBdt.get_ylim()[1]*0.9, r"$t_\textrm{M}$", fontsize=lgd_font_size+2, ha='center')

#Legend
# plt.legend(ncol=2, loc="lower center",  bbox_to_anchor=(0.5, -0.35))
ax_dmBdt.get_legend().remove()

# Draw legend separately
handles_fig_dmBdt, labels_fig_dmBdt = plt.gca().get_legend_handles_labels()
order_fig_dmBdt=[0,1,2]
fig_dmBdt_l, ax_dmBdt_l = plt.subplots()
ax_dmBdt_l.axis(False)
ax_dmBdt_l.legend([handles_fig_dmBdt[i] for i in order_fig_dmBdt],[labels_fig_dmBdt[i] for i in order_fig_dmBdt], loc="center", ncol=3)

plt.show()

fig_dmBdt.savefig("./NP-images/dmBdt.png", dpi=300, bbox_inches='tight', transparent=True)
fig_dmBdt_l.savefig("./NP-images/generic-legend.png", dpi=300, bbox_inches='tight', transparent=True)

## Plot relative growth rate

In [ ]:
'''
Figure: RGR

We used to calculate this, but because of SDs, we read from a file.
'''
#zea_df_rgr_hg = zea_df_m_hg.statted["mean"].diff() / zea_df_m_hg.statted.index.to_series().diff() / zea_df_m_hg.statted["mean"]
#zea_df_rgr_md = zea_df_m_md.statted["mean"].diff() / zea_df_m_md.statted.index.to_series().diff() / zea_df_m_md.statted["mean"]
#zea_df_rgr_lw = zea_df_m_lw.statted["mean"].diff() / zea_df_m_lw.statted.index.to_series().diff() / zea_df_m_lw.statted["mean"]

rgr_lnwt = def_ln_wt
rgr_ls = '--'
rgr_plt_param={'ls':rgr_ls, 'lw':rgr_lnwt}

zea_rgr_df = pd.read_csv(data_path + 'zea_RGR.csv')
zea_rgr_df = zea_rgr_df[zea_rgr_df[time_label] <= harvest_cutoff_zea]
zea_rgr_hg = df_n_selection(zea_rgr_df, nitrogen_label, 1).grouped
zea_rgr_md = df_n_selection(zea_rgr_df, nitrogen_label, 0).grouped
zea_rgr_lw = df_n_selection(zea_rgr_df, nitrogen_label, -1).grouped

zea_rgr_hg[time_label] = zea_rgr_hg[time_label] + point_offset + emerge_offset
zea_rgr_md[time_label] += emerge_offset
zea_rgr_lw[time_label] = zea_rgr_lw[time_label] - point_offset + emerge_offset

# Figure
fig3, ax3 = plt.subplots(1, figsize=(10, 10))
ax3.set_box_aspect(1)

# Draw plots
rgr_lnwt = 5
zea_rgr_hg.plot(x=time_label, y="RGR", yerr="std_RGR", label=hg_label, color=hg_mB_color, **rgr_plt_param, ax=ax3)
zea_rgr_md.plot(x=time_label, y="RGR", yerr="std_RGR", label=md_label, color=md_mB_color, **rgr_plt_param, ax=ax3)
zea_rgr_lw.plot(x=time_label, y="RGR", yerr="std_RGR", label=lw_label, color=lw_mB_color, **rgr_plt_param, ax=ax3)

# Set axes
ax3.set_xlabel(time_xlabel)
# plt.xticks(np.arange(20, harvest_cutoff + 1, 5))
ax3.set_xlim(19.5,40.5)
ax3.xaxis.set_major_locator(ticker.MultipleLocator(5))
ax3.xaxis.set_minor_locator(ticker.MultipleLocator(1))

ax3.set_ylabel("Relative Growth Rate $[\si{\gram\of{DW}\per\day\per\gram\of{DW}}]$")
plt.yticks(np.arange(-0.1, 0.5, 0.1))
ax3.yaxis.set_minor_locator(ticker.MultipleLocator(0.05))

# Draw t_M box and label
ax3.axvspan(30, 35, color="#a5a5a5", alpha=0.25) # label="Harvesting Range"
ax3.text(32.5, ax3.get_ylim()[1]*0.9, r"$t_\textrm{M}$", fontsize=lgd_font_size+2, ha='center')

# Legend
plt.legend(ncol=2, loc="lower center",  bbox_to_anchor=(0.5, -0.35))

ax3.get_legend().remove()

plt.show()

fig3.savefig("./NP-images/RGR.png", dpi=300, bbox_inches='tight', transparent=True)

## Plot nitrogen productivity

In [ ]:
''' 
Figure: Y_N

Let's just use values from the calculation spreadsheet!
'''
# separate settings and finds means and stdevs of Y_N
zea_Y_df = pd.read_csv(data_path + 'zea_Y_N.csv')
zea_Y_df = zea_Y_df[zea_Y_df[time_label] <= harvest_cutoff_zea]
zea_Y_hg = df_n_selection(zea_Y_df, nitrogen_label, 1).grouped
zea_Y_md = df_n_selection(zea_Y_df, nitrogen_label, 0).grouped
zea_Y_lw = df_n_selection(zea_Y_df, nitrogen_label, -1).grouped

zea_Y_hg[time_label] = zea_Y_hg[time_label] + point_offset + emerge_offset
zea_Y_md[time_label] += emerge_offset
zea_Y_lw[time_label] = zea_Y_lw[time_label] - point_offset + emerge_offset

# Y_hg_color="brown"
# Y_md_color="red"
# Y_lw_color="lightcoral"

Y_hg_color=hg_color
Y_md_color=md_color
Y_lw_color=lw_color
Y_LEGEND=False
Y_lnwt=def_ln_wt
Y_args={'ls':'-.', 'lw':Y_lnwt}

# Figure
fig_Y, ax_Y = plt.subplots(1, figsize=(10, 10))
ax_Y.set_box_aspect(1)

# Draw plots

zea_Y_hg.plot(x=time_label, y="Y_N", yerr="std_Y_N", label=hg_label, color=Y_hg_color, **Y_args, ax=ax_Y)
zea_Y_md.plot(x=time_label, y="Y_N", yerr="std_Y_N", label=md_label, color=Y_md_color, **Y_args, ax=ax_Y)
zea_Y_lw.plot(x=time_label, y="Y_N", yerr="std_Y_N", label=lw_label, color=Y_lw_color, **Y_args, ax=ax_Y)
#plt.axhline(y=2.89, label = "Wikstrom 1995", color='grey', linestyle='--')  # lit value

# Set axes
ax_Y.set_xlabel(time_xlabel)
# plt.xticks(np.arange(20,harvest_cutoff,5))
# ax_Y.set_xlim(19.5,harvest_cutoff+0.5)
# ax_Y.xaxis.set_minor_locator(ticker.MultipleLocator(1))
ax_Y.set_xlim(19.5,40.5)
ax_Y.xaxis.set_major_locator(ticker.MultipleLocator(5))
ax_Y.xaxis.set_minor_locator(ticker.MultipleLocator(1))

ax_Y.set_ylabel("Nitrogen Productivity, $\dot{Y}_\mathrm{N}\ [\si{\gram\of{DW}\per\day\per\gram\of{N}}]$")

plt.yticks(np.arange(-1,5,1))
ax_Y.yaxis.set_minor_locator(ticker.MultipleLocator(0.5))
# Not sure why I have to do this here
ax_Y.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))

# Draw t_M box and label
ax_Y.axvspan(30, 35, color="#a5a5a5", alpha=0.25) # label="Harvesting Range"
ax_Y.text(32.5, ax_Y.get_ylim()[1]*0.9, r"$t_\textrm{M}$", fontsize=lgd_font_size+2, ha='center')

# Legend
if Y_LEGEND:
    # handles, labels = plt.gca().get_legend_handles_labels()
    #order = [2,3,4,1,0]
    order = [2,3,1,0]
    # plt.legend([handles[i] for i in order],[labels[i] for i in order], ncol=2, loc="lower center",  bbox_to_anchor=(0.5, -0.35))
    plt.legend(ncol=2, loc="lower center",  bbox_to_anchor=(0.5, -0.35))
else:
    ax_Y.get_legend().remove()

plt.show()

fig_Y.savefig("./NP-images/Y_N.png", dpi=300, bbox_inches='tight', transparent=True)

## Plot N content (mass and percent)

In [ ]:
zea_n_df = pd.read_csv(data_path + 'zea_nitrogen.csv')
zea_n_df = zea_n_df[zea_n_df[time_label] <= harvest_cutoff_zea]

zea_mN_hg = df_n_selection(zea_n_df, nitrogen_label, 1).grouped.dropna()
zea_mN_md = df_n_selection(zea_n_df, nitrogen_label, 0).grouped.dropna()
zea_mN_lw = df_n_selection(zea_n_df, nitrogen_label, -1).grouped.dropna()

zea_mN_hg[time_label] = zea_mN_hg[time_label] + emerge_offset + point_offset
zea_mN_md[time_label] += emerge_offset
zea_mN_lw[time_label] = zea_mN_lw[time_label] + emerge_offset - point_offset

mN_hg_color="MidnightBlue"
mN_md_color="DodgerBlue"
mN_lw_color="SkyBlue"

mN_lnwt = def_ln_wt
mN_ls = '--'
mN_plt_param={'ls':mN_ls, 'lw':mN_lnwt}

pctN_lnwt = def_ln_wt
pctN_ls = '-'
pctN_plt_param={'ls':pctN_ls, 'lw':pctN_lnwt}

# Figure 
fig_mN, ax_mN = plt.subplots(1, figsize=(10, 10))
ax_mN.set_box_aspect(1)

# Draw plots
mN_lnwt = 5
zea_mN_hg.plot(x=time_label, y="mass_N", yerr="SD_mass_N", label=hg_label, color=mN_hg_color, **mN_plt_param, ax=ax_mN)
zea_mN_md.plot(x=time_label, y="mass_N", yerr="SD_mass_N", label=md_label, color=mN_md_color, **mN_plt_param, ax=ax_mN)
zea_mN_lw.plot(x=time_label, y="mass_N", yerr="SD_mass_N", label=lw_label, color=mN_lw_color, **mN_plt_param, ax=ax_mN)

# Set axes
ax_mN.set_xlabel(time_xlabel)
ax_mN.set_xlim(19.5,40.5)
ax_mN.xaxis.set_major_locator(ticker.MultipleLocator(5))
ax_mN.xaxis.set_minor_locator(ticker.MultipleLocator(1))

ax_mN.set_ylabel(r"Plant Nitrogen Content, $m_\text{N}\ [\si{\gram}]$")
ax_mN.yaxis.set_minor_locator(ticker.MultipleLocator(0.05))

# Draw t_M box and label
ax_mN.axvspan(30, 35, color="#a5a5a5", alpha=0.25) # label="Harvesting Range"
ax_mN.text(32.5, ax_mN.get_ylim()[1]*0.9, r"$t_\textrm{M}$", fontsize=lgd_font_size+2, ha='center')

# legend
plt.legend(ncol=2, loc="lower center", bbox_to_anchor=(0.5, -0.35))
ax_mN.get_legend().remove()

plt.show()

# Figure
fig_pctN, ax_pctN = plt.subplots(1, figsize=(10, 10))
ax_pctN.set_box_aspect(1)

# Draw plots
zea_mN_hg.plot(x=time_label, y="pct_N", yerr="SD_pct_N", label=hg_label, color=mN_hg_color, **pctN_plt_param, ax=ax_pctN)
zea_mN_md.plot(x=time_label, y="pct_N", yerr="SD_pct_N", label=md_label, color=mN_md_color, **pctN_plt_param, ax=ax_pctN)
zea_mN_lw.plot(x=time_label, y="pct_N", yerr="SD_pct_N", label=lw_label, color=mN_lw_color, **pctN_plt_param, ax=ax_pctN)

# Set axes
ax_pctN.set_xlabel(time_xlabel)
# ax_pctN.set_xticks(np.arange(20, 41, 5))
ax_pctN.set_xlim(19.5,40.5)
ax_pctN.xaxis.set_major_locator(ticker.MultipleLocator(5))
ax_pctN.xaxis.set_minor_locator(ticker.MultipleLocator(1))

ax_pctN.set_ylabel("Plant Nitrogen Content [\% w/w]")
ax_pctN.yaxis.set_minor_locator(ticker.MultipleLocator(0.05))

# Draw t_M box and label
ax_pctN.axvspan(30, 35, color="#a5a5a5", alpha=0.25) # label="Harvesting Range"
ax_pctN.text(32.5, ax_pctN.get_ylim()[1]*0.9, r"$t_\textrm{M}$", fontsize=lgd_font_size+2, ha='center')

# Legend
# plt.legend(ncol=2, loc="lower center", bbox_to_anchor=(0.5, -0.35))
ax_pctN.get_legend().remove()

plt.show()

fig_mN.savefig("./NP-images/m_N.png", dpi=300, bbox_inches='tight', transparent=True)
fig_pctN.savefig("./NP-images/pct_N.png", dpi=300, bbox_inches='tight', transparent=True)

## Parse NSS measurements

### Set up global params for N plotting

Note: we use NH3 (ammonia) and NH4+ (ammonium) interchangeably for now.

In [ ]:
harvest_cutoff_nitrogen = 38
N_offset = 0
N_time_xlabel = "Time, $t\ [\si{\day\of{AS}}]$"

rc_ylabel = r"Concentration RC, $\underline{\Delta}c$"
conc_ylabel = "Concentration," "\n" r"$c\ [\si{\milli\mole\per\liter}]$"
dc_ylabel = r"Conc. change," "\n" r"$\Delta c / \Delta t\ \left[\si[]{\milli\mole\per\liter\per\day}\right]$"

ln_label = ["Total N","NH$_4^+$","NO$_3^-$"]

room_titlesize = 18
room_lgdsize = 15
room_axissize = 18
room_figsize = (10,10)
room_ln = 5

#nitrate_hg_color="MidnightBlue"
#nitrate_md_color="DodgerBlue"
#nitrate_lw_color="SkyBlue"

#ammonia_hg_color="MidnightBlue"
#ammonia_md_color="DodgerBlue"
#ammonia_lw_color="SkyBlue"

#totalN_hg_color="MidnightBlue"
#totalN_md_color="DodgerBlue"
#totalN_lw_color="SkyBlue"

#scheme1
#nitrate_color="olivedrab"
#ammonia_color="#c72718"
#totalN_color="Blue"

#scheme2
nitrate_color='#283117'  #'#6e8b3d' # "#7f41e1"
ammonia_color='#96BA5B'  #'#52682d' #"#41e1b9"
totalN_color='#607936' # '#96b58d' # "#e18741" #totalN_color="RoyalBlue"
nitrate_ls='--'
ammonia_ls='--'
totalN_ls='-'

nitrate_plt_param={'ls':nitrate_ls, 'color':nitrate_color, 'lw':room_ln, 'dashes':(2.5,1)}
ammonia_plt_param={'ls':ammonia_ls, 'color':ammonia_color, 'lw':room_ln, 'dashes':(1,1.5)}
totalN_plt_param={'ls':totalN_ls, 'color':totalN_color, 'lw':room_ln}

#scheme3
#nitrate_color="#e16141"
#ammonia_color="#a4e141"
#totalN_color="#7f41e1"

### Read in Delta RC

In [ ]:
rc_ylabel = r"Concentration RC, $\underline{\Delta}c$"

rc_label = ["RC_N","RC_NH3","RC_NO3"]
rc_sd_label = ["RC_N_SD", "RC_NH3_SD", "RC_NO3_SD"]

zea_rcn_df = pd.read_csv(data_path + 'zea_RC_N_total_change.csv')
zea_rcn_df = zea_rcn_df[zea_rcn_df[time_label] <= harvest_cutoff_nitrogen]

zea_rcn_hg = df_n_selection(zea_rcn_df, nitrogen_label, 1).grouped
zea_rcn_md = df_n_selection(zea_rcn_df, nitrogen_label, 0).grouped
zea_rcn_lw = df_n_selection(zea_rcn_df, nitrogen_label, -1).grouped

zea_rcn_hg[time_label] += (N_offset)
zea_rcn_md[time_label] += (N_offset)
zea_rcn_lw[time_label] += (N_offset)

zea_rcn_total_hg = zea_rcn_hg[[time_label, rc_label[0], rc_sd_label[0]]].copy()
zea_rcn_total_md = zea_rcn_md[[time_label, rc_label[0], rc_sd_label[0]]].copy()
zea_rcn_total_lw = zea_rcn_lw[[time_label, rc_label[0], rc_sd_label[0]]].copy()

zea_rcn_NH3_hg = zea_rcn_hg[[time_label, rc_label[1], rc_sd_label[1]]].copy()
zea_rcn_NH3_hg[time_label] += point_offset
zea_rcn_NH3_md = zea_rcn_md[[time_label, rc_label[1], rc_sd_label[1]]].copy()
zea_rcn_NH3_md[time_label] += point_offset
zea_rcn_NH3_lw = zea_rcn_lw[[time_label, rc_label[1], rc_sd_label[1]]].copy()
zea_rcn_NH3_lw[time_label] += point_offset

zea_rcn_NO3_hg = zea_rcn_hg[[time_label, rc_label[2], rc_sd_label[2]]].copy()
zea_rcn_NO3_hg[time_label] -= point_offset
zea_rcn_NO3_md = zea_rcn_md[[time_label, rc_label[2], rc_sd_label[2]]].copy()
zea_rcn_NO3_md[time_label] -= point_offset
zea_rcn_NO3_lw = zea_rcn_lw[[time_label, rc_label[2], rc_sd_label[2]]].copy()
zea_rcn_NO3_lw[time_label] -= point_offset

### Read in cN

In [ ]:
c_label = ["c_N","c_NH3","c_NO3"]
c_sd_label = ["c_N_SD", "c_NH3_SD", "c_NO3_SD"]

zea_cn_df = pd.read_csv(data_path + 'zea_cN.csv')
zea_cn_df = zea_cn_df[zea_cn_df[time_label] <= harvest_cutoff]

zea_cn_hg = df_n_selection(zea_cn_df, nitrogen_label, 1).grouped
zea_cn_md = df_n_selection(zea_cn_df, nitrogen_label, 0).grouped
zea_cn_lw = df_n_selection(zea_cn_df, nitrogen_label, -1).grouped

zea_cn_hg[time_label] += (N_offset)
zea_cn_md[time_label] += (N_offset)
zea_cn_lw[time_label] += (N_offset)

zea_cn_total_hg = zea_cn_hg[[time_label, c_label[0], c_sd_label[0]]].copy()
zea_cn_total_md = zea_cn_md[[time_label, c_label[0], c_sd_label[0]]].copy()
zea_cn_total_lw = zea_cn_lw[[time_label, c_label[0], c_sd_label[0]]].copy()

zea_cn_NH3_hg = zea_cn_hg[[time_label, c_label[1], c_sd_label[1]]].copy()
zea_cn_NH3_hg[time_label] += point_offset
zea_cn_NH3_md = zea_cn_md[[time_label, c_label[1], c_sd_label[1]]].copy()
zea_cn_NH3_md[time_label] += point_offset
zea_cn_NH3_lw = zea_cn_lw[[time_label, c_label[1], c_sd_label[1]]].copy()
zea_cn_NH3_lw[time_label] += point_offset

zea_cn_NO3_hg = zea_cn_hg[[time_label, c_label[2], c_sd_label[2]]].copy()
zea_cn_NO3_hg[time_label] -= point_offset
zea_cn_NO3_md = zea_cn_md[[time_label, c_label[2], c_sd_label[2]]].copy()
zea_cn_NO3_md[time_label] -= point_offset
zea_cn_NO3_lw = zea_cn_lw[[time_label, c_label[2], c_sd_label[2]]].copy()
zea_cn_NO3_lw[time_label] -= point_offset

### Read in delta c / delta t

In [ ]:
dc_label = ["dc_N","dc_NH4","dc_NO3"]
dc_sd_label = ["dc_N_SD", "dc_NH4_SD", "dc_NO3_SD"]

zea_dcn_df = pd.read_csv(data_path + 'zea_dcN.csv')
zea_dcn_df = zea_dcn_df[zea_dcn_df[time_label] <= harvest_cutoff]

zea_dcn_hg = df_n_selection(zea_dcn_df, nitrogen_label, 1).grouped
zea_dcn_md = df_n_selection(zea_dcn_df, nitrogen_label, 0).grouped
zea_dcn_lw = df_n_selection(zea_dcn_df, nitrogen_label, -1).grouped

zea_dcn_hg[time_label] += (N_offset)
zea_dcn_md[time_label] += (N_offset)
zea_dcn_lw[time_label] += (N_offset)

zea_dcn_total_hg = zea_dcn_hg[[time_label, dc_label[0], dc_sd_label[0]]].copy()
zea_dcn_total_md = zea_dcn_md[[time_label, dc_label[0], dc_sd_label[0]]].copy()
zea_dcn_total_lw = zea_dcn_lw[[time_label, dc_label[0], dc_sd_label[0]]].copy()

zea_dcn_NH3_hg = zea_dcn_hg[[time_label, dc_label[1], dc_sd_label[1]]].copy()
zea_dcn_NH3_hg[time_label] += point_offset
zea_dcn_NH3_md = zea_dcn_md[[time_label, dc_label[1], dc_sd_label[1]]].copy()
zea_dcn_NH3_md[time_label] += point_offset
zea_dcn_NH3_lw = zea_dcn_lw[[time_label, dc_label[1], dc_sd_label[1]]].copy()
zea_dcn_NH3_lw[time_label] += point_offset

zea_dcn_NO3_hg = zea_dcn_hg[[time_label, dc_label[2], dc_sd_label[2]]].copy()
zea_dcn_NO3_hg[time_label] -= point_offset
zea_dcn_NO3_md = zea_dcn_md[[time_label, dc_label[2], dc_sd_label[2]]].copy()
zea_dcn_NO3_md[time_label] -= point_offset
zea_dcn_NO3_lw = zea_dcn_lw[[time_label, dc_label[2], dc_sd_label[2]]].copy()
zea_dcn_NO3_lw[time_label] -= point_offset

## individual NSS $\Delta$RC plots

In [ ]:
rcn_y_maj_tk = 0.25
rcn_y_min_tk = 0.125
rcn_x_maj_tk = 5
rcn_x_min_tk = 1

ax_rcn_hg = zea_rcn_total_hg.plot(x=time_label, y=rc_label[0], yerr=rc_sd_label[0], label=ln_label[0], figsize=room_figsize, **totalN_plt_param)
zea_rcn_NH3_hg.plot(x=time_label, y=rc_label[1], yerr=rc_sd_label[1], label=ln_label[1], ax=ax_rcn_hg, **ammonia_plt_param)
zea_rcn_NO3_hg.plot(x=time_label, y=rc_label[2], yerr=rc_sd_label[2], label=ln_label[2], ax=ax_rcn_hg, **nitrate_plt_param)

ax_rcn_hg.get_legend().remove()
ax_rcn_hg.set_xlabel(N_time_xlabel)
ax_rcn_hg.set_ylabel(rc_ylabel)
ax_rcn_hg.set_title("Excess")

plt.xlim(4.5,40.5)
plt.ylim(-1.05,0.4)
ax_rcn_hg.yaxis.set_major_locator(ticker.MultipleLocator(rcn_y_maj_tk))
ax_rcn_hg.yaxis.set_minor_locator(ticker.MultipleLocator(rcn_y_min_tk))
ax_rcn_hg.xaxis.set_major_locator(ticker.MultipleLocator(rcn_x_maj_tk))
ax_rcn_hg.xaxis.set_minor_locator(ticker.MultipleLocator(rcn_x_min_tk))
plt.savefig("./NP-images/cN-RC_excess.png", dpi=300, bbox_inches='tight', transparent=True)

ax_rcn_md = zea_rcn_total_md.plot(x=time_label, y=rc_label[0], yerr=rc_sd_label[0], label=ln_label[0], figsize=room_figsize, **totalN_plt_param)
zea_rcn_NH3_md.plot(x=time_label, y=rc_label[1], yerr=rc_sd_label[1], label=ln_label[1], ax=ax_rcn_md, **ammonia_plt_param)
zea_rcn_NO3_md.plot(x=time_label, y=rc_label[2], yerr=rc_sd_label[2], label=ln_label[2], ax=ax_rcn_md, **nitrate_plt_param)

ax_rcn_md.get_legend().remove()
ax_rcn_md.set_xlabel(N_time_xlabel)
ax_rcn_md.set_ylabel(rc_ylabel)
ax_rcn_md.set_title("Normal")

plt.xlim(4.5,40.5)
plt.ylim(-1.05,0.4)
ax_rcn_md.yaxis.set_major_locator(ticker.MultipleLocator(rcn_y_maj_tk))
ax_rcn_md.yaxis.set_minor_locator(ticker.MultipleLocator(rcn_y_min_tk))
ax_rcn_md.xaxis.set_major_locator(ticker.MultipleLocator(rcn_x_maj_tk))
ax_rcn_md.xaxis.set_minor_locator(ticker.MultipleLocator(rcn_x_min_tk))
plt.savefig("./NP-images/cN-RC_normal.png", dpi=300, bbox_inches='tight', transparent=True)

ax_rcn_lw = zea_rcn_total_lw.plot(x=time_label, y=rc_label[0], yerr=rc_sd_label[0], label=ln_label[0], figsize=room_figsize, **totalN_plt_param)
zea_rcn_NH3_lw.plot(x=time_label, y=rc_label[1], yerr=rc_sd_label[1], label=ln_label[1], ax=ax_rcn_lw, **ammonia_plt_param)
zea_rcn_NO3_lw.plot(x=time_label, y=rc_label[2], yerr=rc_sd_label[2], label=ln_label[2], ax=ax_rcn_lw, **nitrate_plt_param)

ax_rcn_lw.get_legend().remove()
ax_rcn_lw.set_xlabel(N_time_xlabel)
ax_rcn_lw.set_ylabel(rc_ylabel)
ax_rcn_lw.set_title("Deficient")

plt.xlim(4.5,40.5)
plt.ylim(-1.05,0.4)
ax_rcn_lw.yaxis.set_major_locator(ticker.MultipleLocator(rcn_y_maj_tk))
ax_rcn_lw.yaxis.set_minor_locator(ticker.MultipleLocator(rcn_y_min_tk))
ax_rcn_lw.xaxis.set_major_locator(ticker.MultipleLocator(rcn_x_maj_tk))
ax_rcn_lw.xaxis.set_minor_locator(ticker.MultipleLocator(rcn_x_min_tk))
plt.savefig("./NP-images/cN-RC_deficient.png", dpi=300, bbox_inches='tight', transparent=True)

# Draw legend separately
handles, labels = plt.gca().get_legend_handles_labels()
order=[0,1,2]
fig_rcn_l, ax_rcn_lw_l = plt.subplots()
ax_rcn_lw_l.axis(False)
ax_rcn_lw_l.legend([handles[i] for i in order],[labels[i] for i in order], loc="center", ncol=3)

plt.savefig("./NP-images/cN-RC_legend.png", dpi=300, bbox_inches='tight', transparent=True)

plt.show()


## Individual NSS conc plots

In [ ]:
cn_y_maj_tk = 2.5
cn_y_min_tk = 1.25
cn_x_maj_tk = 5.0
cn_x_min_tk = 1.0

ax_c_hg = zea_cn_total_hg.plot(x=time_label, y=c_label[0], yerr=c_sd_label[0], figsize=room_figsize, **totalN_plt_param)
zea_cn_NH3_hg.plot(x=time_label, y=c_label[1], yerr=c_sd_label[1], figsize=room_figsize, ax=ax_c_hg, **ammonia_plt_param)
zea_cn_NO3_hg.plot(x=time_label, y=c_label[2], yerr=c_sd_label[2], figsize=room_figsize, ax=ax_c_hg, **nitrate_plt_param)

ax_c_hg.get_legend().remove()
ax_c_hg.set_xlabel(N_time_xlabel)
ax_c_hg.set_ylabel(conc_ylabel)
ax_c_hg.set_title("Excess")

plt.xlim(-0.5,40.5)
plt.ylim(-0.5,15)
ax_c_hg.yaxis.set_major_locator(ticker.MultipleLocator(cn_y_maj_tk))
ax_c_hg.yaxis.set_minor_locator(ticker.MultipleLocator(cn_y_min_tk))
ax_c_hg.xaxis.set_major_locator(ticker.MultipleLocator(cn_x_maj_tk))
ax_c_hg.xaxis.set_minor_locator(ticker.MultipleLocator(cn_x_min_tk))
plt.savefig("./NP-images/cN-c_excess.png", dpi=300, bbox_inches='tight', transparent=True)

ax_c_md = zea_cn_total_md.plot(x=time_label, y=c_label[0], yerr=c_sd_label[0], figsize=room_figsize, **totalN_plt_param)
zea_cn_NH3_md.plot(x=time_label, y=c_label[1], yerr=c_sd_label[1], figsize=room_figsize, ax=ax_c_md, **ammonia_plt_param)
zea_cn_NO3_md.plot(x=time_label, y=c_label[2], yerr=c_sd_label[2], figsize=room_figsize, ax=ax_c_md, **nitrate_plt_param)

ax_c_md.get_legend().remove()
ax_c_md.set_xlabel(N_time_xlabel)
ax_c_md.set_ylabel(conc_ylabel)
ax_c_md.set_title("Normal")

plt.xticks(np.arange(3.5, 40.5, 10))
plt.xlim(-0.5,40.5)
plt.ylim(-0.5,15)
ax_c_md.yaxis.set_major_locator(ticker.MultipleLocator(cn_y_maj_tk))
ax_c_md.yaxis.set_minor_locator(ticker.MultipleLocator(cn_y_min_tk))
ax_c_md.xaxis.set_major_locator(ticker.MultipleLocator(cn_x_maj_tk))
ax_c_md.xaxis.set_minor_locator(ticker.MultipleLocator(cn_x_min_tk))
plt.savefig("./NP-images/cN-c_normal.png", dpi=300, bbox_inches='tight', transparent=True)

ax_c_lw = zea_cn_total_lw.plot(x=time_label, y=c_label[0], yerr=c_sd_label[0], figsize=room_figsize, **totalN_plt_param)
zea_cn_NH3_lw.plot(x=time_label, y=c_label[1], yerr=c_sd_label[1], figsize=room_figsize, ax=ax_c_lw, **ammonia_plt_param)
zea_cn_NO3_lw.plot(x=time_label, y=c_label[2], yerr=c_sd_label[2], figsize=room_figsize, ax=ax_c_lw, **nitrate_plt_param)

ax_c_lw.get_legend().remove()
ax_c_lw.set_xlabel(N_time_xlabel)
ax_c_lw.set_ylabel(conc_ylabel)
ax_c_lw.set_title("Deficient")

plt.xlim(-0.5,40.5)
plt.ylim(-0.5,15)
ax_c_lw.yaxis.set_major_locator(ticker.MultipleLocator(cn_y_maj_tk))
ax_c_lw.yaxis.set_minor_locator(ticker.MultipleLocator(cn_y_min_tk))
ax_c_lw.xaxis.set_major_locator(ticker.MultipleLocator(cn_x_maj_tk))
ax_c_lw.xaxis.set_minor_locator(ticker.MultipleLocator(cn_x_min_tk))
plt.savefig("./NP-images/cN-c_deficient.png", dpi=300, bbox_inches='tight', transparent=True)

plt.show()


## c and dc/dt

In [ ]:
dcn_x_maj_tk = 5.0
dcn_x_min_tk = 1.0

dcn_ax_title_size = 36
dcn_title_size = 24
dcn_label_size = 36
DCN_LEGEND = False

# d_nitrate_plt_param={'ls':nitrate_ls, 'color':nitrate_color, 'lw':room_ln, 'dashes':(2.5,1)}
# d_ammonia_plt_param={'ls':ammonia_ls, 'color':ammonia_color, 'lw':room_ln, 'dashes':(1,1.5)}
# d_totalN_plt_param={'ls':totalN_ls, 'color':totalN_color, 'lw':room_ln, 'fontsize': dcn_}

dcn_y0_hg_maj_tk = 5.0
dcn_y0_hg_min_tk = 2.5
dcn_y1_hg_maj_tk = 0.25
dcn_y1_hg_min_tk = 0.125

fig_dcn_hg, ax_dcn_hg = plt.subplots(2, sharex=True, figsize=(10,10))

zea_cn_total_hg.plot(x=time_label, y=c_label[0], yerr=c_sd_label[0], ax=ax_dcn_hg[0], **totalN_plt_param)
zea_cn_NH3_hg.plot(x=time_label, y=c_label[1], yerr=c_sd_label[1], ax=ax_dcn_hg[0], **ammonia_plt_param)
zea_cn_NO3_hg.plot(x=time_label, y=c_label[2], yerr=c_sd_label[2], ax=ax_dcn_hg[0], **nitrate_plt_param)

zea_dcn_total_hg.plot(x=time_label, y=dc_label[0], yerr=dc_sd_label[0], figsize=room_figsize, ax=ax_dcn_hg[1], **totalN_plt_param)
zea_dcn_NH3_hg.plot(x=time_label, y=dc_label[1], yerr=dc_sd_label[1], figsize=room_figsize, ax=ax_dcn_hg[1], **ammonia_plt_param)
zea_dcn_NO3_hg.plot(x=time_label, y=dc_label[2], yerr=dc_sd_label[2], figsize=room_figsize, ax=ax_dcn_hg[1], **nitrate_plt_param)

# fig_dcn_hg.suptitle("Excess", fontsize=dcn_title_size)

# ax_dcn_hg[1].set_ylabel(dc_ylabel, fontsize=dcn_ax_title_size)
ax_dcn_hg[1].yaxis.set_major_locator(ticker.MultipleLocator(dcn_y1_hg_maj_tk))
ax_dcn_hg[1].yaxis.set_minor_locator(ticker.MultipleLocator(dcn_y1_hg_min_tk))
ax_dcn_hg[1].set_ylim(-0.4,0.4)
ax_dcn_hg[1].tick_params(axis='both', which='major', labelsize=dcn_label_size)

# ax_dcn_hg[0].set_ylabel(conc_ylabel, fontsize=dcn_ax_title_size)
ax_dcn_hg[0].yaxis.set_major_locator(ticker.MultipleLocator(dcn_y0_hg_maj_tk))
ax_dcn_hg[0].yaxis.set_minor_locator(ticker.MultipleLocator(dcn_y0_hg_min_tk))
ax_dcn_hg[0].yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
ax_dcn_hg[0].set_ylim(0,15)
ax_dcn_hg[0].tick_params(axis='both', which='major', labelsize=dcn_label_size)

ax_dcn_hg[1].set_xlabel(N_time_xlabel, fontsize=dcn_ax_title_size)
ax_dcn_hg[1].xaxis.set_major_locator(ticker.MultipleLocator(dcn_x_maj_tk))
ax_dcn_hg[1].xaxis.set_minor_locator(ticker.MultipleLocator(dcn_x_min_tk))

fig_dcn_hg.align_ylabels(ax_dcn_hg)

if not DCN_LEGEND:
    ax_dcn_hg[0].get_legend().remove()
    ax_dcn_hg[1].get_legend().remove()

plt.savefig("./NP-images/cN_dcN_excess.png", dpi=300, bbox_inches='tight', transparent=True)

dcn_y0_md_maj_tk = 5.0
dcn_y0_md_min_tk = 2.5
dcn_y1_md_maj_tk = 0.25
dcn_y1_md_min_tk = 0.125

fig_dcn_md, ax_dcn_md = plt.subplots(2, sharex=True, figsize=(10,10))

zea_cn_total_md.plot(x=time_label, y=c_label[0], yerr=c_sd_label[0], ax=ax_dcn_md[0], **totalN_plt_param)
zea_cn_NH3_md.plot(x=time_label, y=c_label[1], yerr=c_sd_label[1], ax=ax_dcn_md[0], **ammonia_plt_param)
zea_cn_NO3_md.plot(x=time_label, y=c_label[2], yerr=c_sd_label[2], ax=ax_dcn_md[0], **nitrate_plt_param)

zea_dcn_total_md.plot(x=time_label, y=dc_label[0], yerr=dc_sd_label[0], ax=ax_dcn_md[1], **totalN_plt_param)
zea_dcn_NH3_md.plot(x=time_label, y=dc_label[1], yerr=dc_sd_label[1], ax=ax_dcn_md[1], **ammonia_plt_param)
zea_dcn_NO3_md.plot(x=time_label, y=dc_label[2], yerr=dc_sd_label[2], ax=ax_dcn_md[1], **nitrate_plt_param)

# fig_dcn_md.suptitle("Normal", fontsize=dcn_title_size)

# ax_dcn_md[1].set_ylabel(dc_ylabel, fontsize=dcn_ax_title_size)
ax_dcn_md[1].yaxis.set_major_locator(ticker.MultipleLocator(dcn_y1_md_maj_tk))
ax_dcn_md[1].yaxis.set_minor_locator(ticker.MultipleLocator(dcn_y1_md_min_tk))
ax_dcn_md[1].set_ylim(-0.4,0.4)
ax_dcn_md[1].tick_params(axis='both', which='major', labelsize=dcn_label_size)

# ax_dcn_md[0].set_ylabel(conc_ylabel, fontsize=dcn_ax_title_size)
ax_dcn_md[0].yaxis.set_major_locator(ticker.MultipleLocator(dcn_y0_md_maj_tk))
ax_dcn_md[0].yaxis.set_minor_locator(ticker.MultipleLocator(dcn_y0_md_min_tk))
ax_dcn_md[0].yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
ax_dcn_md[0].set_ylim(0,15)
ax_dcn_md[0].tick_params(axis='both', which='major', labelsize=dcn_label_size)

ax_dcn_md[1].set_xlabel(N_time_xlabel, fontsize=dcn_ax_title_size)
ax_dcn_md[1].xaxis.set_major_locator(ticker.MultipleLocator(dcn_x_maj_tk))
ax_dcn_md[1].xaxis.set_minor_locator(ticker.MultipleLocator(dcn_x_min_tk))

fig_dcn_md.align_ylabels(ax_dcn_md)

if not DCN_LEGEND:
    ax_dcn_md[0].get_legend().remove()
    ax_dcn_md[1].get_legend().remove()

plt.savefig("./NP-images/cN_dcN_normal.png", dpi=300, bbox_inches='tight', transparent=True)

dcn_y0_lw_maj_tk = 5.0
dcn_y0_lw_min_tk = 2.5
dcn_y1_lw_maj_tk = 0.25
dcn_y1_lw_min_tk = 0.125

fig_dcn_lw, ax_dcn_lw = plt.subplots(2, sharex=True, figsize=(10,10))

zea_cn_total_lw.plot(x=time_label, y=c_label[0], yerr=c_sd_label[0], ax=ax_dcn_lw[0], **totalN_plt_param)
zea_cn_NH3_lw.plot(x=time_label, y=c_label[1], yerr=c_sd_label[1], ax=ax_dcn_lw[0], **ammonia_plt_param)
zea_cn_NO3_lw.plot(x=time_label, y=c_label[2], yerr=c_sd_label[2], ax=ax_dcn_lw[0], **nitrate_plt_param)

zea_dcn_total_lw.plot(x=time_label, y=dc_label[0], yerr=dc_sd_label[0], ax=ax_dcn_lw[1], **totalN_plt_param)
zea_dcn_NH3_lw.plot(x=time_label, y=dc_label[1], yerr=dc_sd_label[1], ax=ax_dcn_lw[1], **ammonia_plt_param)
zea_dcn_NO3_lw.plot(x=time_label, y=dc_label[2], yerr=dc_sd_label[2], ax=ax_dcn_lw[1], **nitrate_plt_param)

# fig_dcn_lw.suptitle("Deficient", fontsize=dcn_title_size)

ax_dcn_lw[1].set_ylabel(dc_ylabel, fontsize=dcn_ax_title_size)
ax_dcn_lw[1].yaxis.set_major_locator(ticker.MultipleLocator(dcn_y1_lw_maj_tk))
ax_dcn_lw[1].yaxis.set_minor_locator(ticker.MultipleLocator(dcn_y1_lw_min_tk))
ax_dcn_lw[1].set_ylim(-0.4,0.4)
ax_dcn_lw[1].tick_params(axis='both', which='major', labelsize=dcn_label_size)

ax_dcn_lw[0].set_ylabel(conc_ylabel, fontsize=dcn_ax_title_size)
ax_dcn_lw[0].yaxis.set_major_locator(ticker.MultipleLocator(dcn_y0_lw_maj_tk))
ax_dcn_lw[0].yaxis.set_minor_locator(ticker.MultipleLocator(dcn_y0_lw_min_tk))
ax_dcn_lw[0].yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
ax_dcn_lw[0].set_ylim(0,15)
ax_dcn_lw[0].tick_params(axis='both', which='major', labelsize=dcn_label_size)

ax_dcn_lw[1].set_xlabel(N_time_xlabel, fontsize=dcn_ax_title_size)
ax_dcn_lw[1].xaxis.set_major_locator(ticker.MultipleLocator(dcn_x_maj_tk))
ax_dcn_lw[1].xaxis.set_minor_locator(ticker.MultipleLocator(dcn_x_min_tk))

fig_dcn_lw.align_ylabels(ax_dcn_lw)

if not DCN_LEGEND:
    ax_dcn_lw[0].get_legend().remove()
    ax_dcn_lw[1].get_legend().remove()

plt.savefig("./NP-images/cN_dcN_deficient.png", dpi=300, bbox_inches='tight', transparent=True)

plt.show()

In [ ]:
# # Plotting both cN and dcN was cluttered

# cn_y_maj_tk = 2.5
# cn_y_min_tk = 1.25
# cn_x_maj_tk = 5
# cn_x_min_tk = 1

# ax_c_hg = zea_cn_total_hg.plot(x=time_label, y=c_label[0], yerr=c_sd_label[0], figsize=room_figsize, **totalN_plt_param)
# zea_cn_NH3_hg.plot(x=time_label, y=c_label[1], yerr=c_sd_label[1], figsize=room_figsize, ax=ax_c_hg, **ammonia_plt_param)
# zea_cn_NO3_hg.plot(x=time_label, y=c_label[2], yerr=c_sd_label[2], figsize=room_figsize, ax=ax_c_hg, **nitrate_plt_param)

# ax_dcn_hg = ax_c_hg.twinx()
# zea_dcn_total_hg.plot(x=time_label, y=dc_label[0], yerr=dc_sd_label[0], figsize=room_figsize, ax=ax_dcn_hg, **totalN_plt_param)
# zea_dcn_NH3_hg.plot(x=time_label, y=dc_label[1], yerr=dc_sd_label[1], figsize=room_figsize, ax=ax_dcn_hg, **ammonia_plt_param)
# zea_dcn_NO3_hg.plot(x=time_label, y=dc_label[2], yerr=dc_sd_label[2], figsize=room_figsize, ax=ax_dcn_hg, **nitrate_plt_param)


# ax_c_hg.get_legend().remove()
# ax_c_hg.set_xlabel(N_time_xlabel)
# ax_c_hg.set_ylabel(conc_ylabel)
# ax_c_hg.set_title("Excess")

# # plt.xlim(-0.5,40.5)
# # plt.ylim(-0.5,15)
# ax_c_hg.yaxis.set_major_locator(ticker.MultipleLocator(cn_y_maj_tk))
# ax_c_hg.yaxis.set_minor_locator(ticker.MultipleLocator(cn_y_min_tk))
# ax_c_hg.xaxis.set_major_locator(ticker.MultipleLocator(cn_x_maj_tk))
# ax_c_hg.xaxis.set_minor_locator(ticker.MultipleLocator(cn_x_min_tk))
# # plt.savefig("./NP-images/x_excess.png", dpi=300, bbox_inches='tight', transparent=True)

# plt.show()

## Individual plots by N form

In [ ]:
cNO3_y_maj_tk = 0.5
cNO3_y_min_tk = 0.25

'''
We need to reload this stuff so that there is no offset
'''
zea_cn_total_hg = zea_cn_hg[[time_label, c_label[0], c_sd_label[0]]].copy()
zea_cn_total_md = zea_cn_md[[time_label, c_label[0], c_sd_label[0]]].copy()
zea_cn_total_lw = zea_cn_lw[[time_label, c_label[0], c_sd_label[0]]].copy()

zea_cn_NH3_hg = zea_cn_hg[[time_label, c_label[1], c_sd_label[1]]].copy()
zea_cn_NH3_md = zea_cn_md[[time_label, c_label[1], c_sd_label[1]]].copy()
zea_cn_NH3_lw = zea_cn_lw[[time_label, c_label[1], c_sd_label[1]]].copy()

zea_cn_NO3_hg = zea_cn_hg[[time_label, c_label[2], c_sd_label[2]]].copy()
zea_cn_NO3_md = zea_cn_md[[time_label, c_label[2], c_sd_label[2]]].copy()
zea_cn_NO3_lw = zea_cn_lw[[time_label, c_label[2], c_sd_label[2]]].copy()

ammonia_plt_param2={'ls':ammonia_ls, 'lw':room_ln, 'dashes':(1,1.5)}
nitrate_plt_param2={'ls':nitrate_ls, 'lw':room_ln, 'dashes':(2.5,1)}
totalN_plt_param2={'ls':totalN_ls, 'lw':room_ln}

ax_total_c = zea_cn_total_hg.plot(x=time_label, y=c_label[0], yerr=c_sd_label[0], label="Excess", figsize=room_figsize, color=hg_mB_color, **totalN_plt_param2)
zea_cn_total_md.plot(x=time_label, y=c_label[0], yerr=c_sd_label[0], label="Normal", figsize=room_figsize, ax=ax_total_c, color=md_mB_color, **totalN_plt_param2)
zea_cn_total_lw.plot(x=time_label, y=c_label[0], yerr=c_sd_label[0], label="Deficient", figsize=room_figsize, ax=ax_total_c, color=lw_mB_color, **totalN_plt_param2)

handles, labels = ax_total_c.get_legend_handles_labels()
ax_total_c.legend(reversed(handles), reversed(labels), ncol=3)
ax_total_c.set_xlabel(N_time_xlabel)
ax_total_c.set_ylabel(conc_ylabel)
ax_total_c.set_title("Total Nitrogen")

plt.xlim(-0.5,40.5)
plt.ylim(-0.5,15)
ax_total_c.yaxis.set_major_locator(ticker.MultipleLocator(cn_y_maj_tk))
ax_total_c.yaxis.set_minor_locator(ticker.MultipleLocator(cn_y_min_tk))
ax_total_c.xaxis.set_major_locator(ticker.MultipleLocator(cn_x_maj_tk))
ax_total_c.xaxis.set_minor_locator(ticker.MultipleLocator(cn_x_min_tk))
plt.savefig("./NP-images/c_total.png", dpi=300, bbox_inches='tight', transparent=True)

ax_amm_c = zea_cn_NH3_hg.plot(x=time_label, y=c_label[1], yerr=c_sd_label[1], label="Excess", figsize=room_figsize, color=hg_mB_color, **ammonia_plt_param2)
zea_cn_NH3_md.plot(x=time_label, y=c_label[1], yerr=c_sd_label[1], label="Normal", figsize=room_figsize, ax=ax_amm_c, color=md_mB_color, **ammonia_plt_param2)
zea_cn_NH3_lw.plot(x=time_label, y=c_label[1], yerr=c_sd_label[1], label="Deficient", figsize=room_figsize, ax=ax_amm_c, color=lw_mB_color, **ammonia_plt_param2)

handles, labels = ax_amm_c.get_legend_handles_labels()
ax_amm_c.legend(reversed(handles), reversed(labels), ncol=3)
ax_amm_c.set_xlabel(N_time_xlabel)
ax_amm_c.set_ylabel(conc_ylabel)
ax_amm_c.set_title("Ammonium")

plt.xlim(-0.5,40.5)
plt.ylim(-0.5,15)
ax_amm_c.yaxis.set_major_locator(ticker.MultipleLocator(cn_y_maj_tk))
ax_amm_c.yaxis.set_minor_locator(ticker.MultipleLocator(cn_y_min_tk))
ax_amm_c.xaxis.set_major_locator(ticker.MultipleLocator(cn_x_maj_tk))
ax_amm_c.xaxis.set_minor_locator(ticker.MultipleLocator(cn_x_min_tk))
plt.savefig("./NP-images/c_ammonium.png", dpi=300, bbox_inches='tight', transparent=True)

ax_NO3_c = zea_cn_NO3_hg.plot(x=time_label, y=c_label[2], yerr=c_sd_label[2], label="Excess", figsize=room_figsize, color=hg_mB_color, **nitrate_plt_param2)
zea_cn_NO3_md.plot(x=time_label, y=c_label[2], yerr=c_sd_label[2], label="Normal", figsize=room_figsize, ax=ax_NO3_c, color=md_mB_color, **nitrate_plt_param2)
zea_cn_NO3_lw.plot(x=time_label, y=c_label[2], yerr=c_sd_label[2], label="Deficient", figsize=room_figsize, ax=ax_NO3_c, color=lw_mB_color, **nitrate_plt_param2)

handles, labels = ax_NO3_c.get_legend_handles_labels()
ax_NO3_c.legend(reversed(handles), reversed(labels), ncol=3)
ax_NO3_c.set_xlabel(N_time_xlabel)
ax_NO3_c.set_ylabel(conc_ylabel)
ax_NO3_c.set_title("Nitrate")

plt.xlim(-0.5,40.5)
plt.ylim(-0.05,2)
ax_NO3_c.yaxis.set_major_locator(ticker.MultipleLocator(cNO3_y_maj_tk))
ax_NO3_c.yaxis.set_minor_locator(ticker.MultipleLocator(cNO3_y_min_tk))
ax_NO3_c.xaxis.set_major_locator(ticker.MultipleLocator(cn_x_maj_tk))
ax_NO3_c.xaxis.set_minor_locator(ticker.MultipleLocator(cn_x_min_tk))
plt.savefig("./NP-images/c_nitrate.png", dpi=300, bbox_inches='tight', transparent=True)

plt.show()

## combined NSS plots

In [ ]:
# nss_rc_fig, nss_rc_axes = plt.subplots(nrows=3, ncols=1, sharex=True, sharey=True)

# nss_rc_fig.set_figheight(15)
# nss_rc_fig.set_figwidth(5)

# for ax in nss_rc_axes:
#     ax.set_adjustable('box', share=True)
#     ax.xaxis.set_minor_locator(ticker.MultipleLocator(5))

# nss_rc_axes[0].errorbar(zea_rcn_total_hg[time_label], zea_rcn_total_hg[rc_label[0]], yerr=zea_rcn_total_hg[rc_sd_label[0]], label=ln_label[0], color=totalN_color, zorder=3)
# nss_rc_axes[0].errorbar(zea_rcn_NH3_hg[time_label], zea_rcn_NH3_hg[rc_label[1]], yerr=zea_rcn_NH3_hg[rc_sd_label[1]], label=ln_label[1], color=ammonia_color, ls='--')
# nss_rc_axes[0].errorbar(zea_rcn_NO3_hg[time_label], zea_rcn_NO3_hg[rc_label[2]], yerr=zea_rcn_NO3_hg[rc_sd_label[2]], label=ln_label[2], color=nitrate_color, ls='--')
# nss_rc_axes[0].set_title("Excess Nitrogen Treatment")
# nss_rc_axes[1].errorbar(zea_rcn_total_md[time_label], zea_rcn_total_md[rc_label[0]], yerr=zea_rcn_total_md[rc_sd_label[0]], label=ln_label[0], color=totalN_color, zorder=3)
# nss_rc_axes[1].errorbar(zea_rcn_NH3_md[time_label], zea_rcn_NH3_md[rc_label[1]], yerr=zea_rcn_NH3_md[rc_sd_label[1]], label=ln_label[1], color=ammonia_color, ls='--')
# nss_rc_axes[1].errorbar(zea_rcn_NO3_md[time_label], zea_rcn_NO3_md[rc_label[2]], yerr=zea_rcn_NO3_md[rc_sd_label[2]], label=ln_label[2], color=nitrate_color, ls='--')
# nss_rc_axes[1].set_title("Normal Nitrogen Treatment")
# nss_rc_axes[2].errorbar(zea_rcn_total_lw[time_label], zea_rcn_total_lw[rc_label[0]], yerr=zea_rcn_total_lw[rc_sd_label[0]], label=ln_label[0], color=totalN_color, zorder=3)
# nss_rc_axes[2].errorbar(zea_rcn_NH3_lw[time_label], zea_rcn_NH3_lw[rc_label[1]], yerr=zea_rcn_NH3_lw[rc_sd_label[1]], label=ln_label[1], color=ammonia_color, ls='--')
# nss_rc_axes[2].errorbar(zea_rcn_NO3_lw[time_label], zea_rcn_NO3_lw[rc_label[2]], yerr=zea_rcn_NO3_lw[rc_sd_label[2]], label=ln_label[2], color=nitrate_color, ls='--')
# nss_rc_axes[2].set_title("Deficient Nitrogen Treatment")

# nss_rc_fig.supxlabel(N_time_xlabel)
# nss_rc_fig.supylabel(rc_ylabel, x=-0.2)

# nss_rc_fig.subplots_adjust(bottom=0.1)  # we need these hax so the legend doesn't go on top of x label
# nss_rc_fig.legend(labels=["Total N","NH$_3$","NO$_3^-$"], ncol=3, loc="lower center",  bbox_to_anchor=(0.5, -0.05))

# plt.xticks(np.arange(10, 41, 10))
# plt.show()

In [ ]:
# '''
# As above, but horizontal
# '''
# room_fig, room_axes = plt.subplots(nrows=1, ncols=3, sharex=True, sharey=True)

# room_fig.set_figheight(5)
# room_fig.set_figwidth(15)

# room_titlesize = plt_font_size
# room_lgdsize = lgd_font_size
# room_axissize = plt_font_size

# for ax in room_axes:
#     # ax.set_adjustable('box', share=True)
#     ax.xaxis.set_minor_locator(ticker.MultipleLocator(5))
#     # ax.tick_params(axis='both', labelsize=room_axissize)
#     ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.25))

# room_axes[2].errorbar(zea_rcn_total_hg[time_label], zea_rcn_total_hg[rc_label[0]], yerr=zea_rcn_total_hg[rc_sd_label[0]], label=ln_label[0], color=totalN_color, ls='--', zorder=3)
# room_axes[2].errorbar(zea_rcn_NH3_hg[time_label], zea_rcn_NH3_hg[rc_label[1]], yerr=zea_rcn_NH3_hg[rc_sd_label[1]], label=ln_label[1], color=ammonia_color, ls='-')
# room_axes[2].errorbar(zea_rcn_NO3_hg[time_label], zea_rcn_NO3_hg[rc_label[2]], yerr=zea_rcn_NO3_hg[rc_sd_label[2]], label=ln_label[2], color=nitrate_color, ls='-')
# room_axes[2].set_title("Excess", fontsize=room_titlesize)
# room_axes[1].errorbar(zea_rcn_total_md[time_label], zea_rcn_total_md[rc_label[0]], yerr=zea_rcn_total_md[rc_sd_label[0]], label=ln_label[0], color=totalN_color, ls='--', zorder=3)
# room_axes[1].errorbar(zea_rcn_NH3_md[time_label], zea_rcn_NH3_md[rc_label[1]], yerr=zea_rcn_NH3_md[rc_sd_label[1]], label=ln_label[1], color=ammonia_color, ls='-')
# room_axes[1].errorbar(zea_rcn_NO3_md[time_label], zea_rcn_NO3_md[rc_label[2]], yerr=zea_rcn_NO3_md[rc_sd_label[2]], label=ln_label[2], color=nitrate_color, ls='-')
# room_axes[1].set_title("Normal", fontsize=room_titlesize)
# room_axes[0].errorbar(zea_rcn_total_lw[time_label], zea_rcn_total_lw[rc_label[0]], yerr=zea_rcn_total_lw[rc_sd_label[0]], label=ln_label[0], color=totalN_color, ls='--', zorder=3)
# room_axes[0].errorbar(zea_rcn_NH3_lw[time_label], zea_rcn_NH3_lw[rc_label[1]], yerr=zea_rcn_NH3_lw[rc_sd_label[1]], label=ln_label[1], color=ammonia_color, ls='-')
# room_axes[0].errorbar(zea_rcn_NO3_lw[time_label], zea_rcn_NO3_lw[rc_label[2]], yerr=zea_rcn_NO3_lw[rc_sd_label[2]], label=ln_label[2], color=nitrate_color, ls='-')
# room_axes[0].set_title("Deficient", fontsize=room_titlesize)

# room_fig.supxlabel(N_time_xlabel, fontsize=room_axissize)
# room_fig.supylabel(r"Concentration RC, $\underline{\Delta}c$", ha='center', fontsize=room_axissize)

# room_fig.subplots_adjust(bottom=0.25)  # we need these hax so the legend doesn't go on top of x label
# room_fig.legend(labels=["Total N","NH$_3$","NO$_3^-$"], ncol=3, loc="lower center",  bbox_to_anchor=(0.5, -0.15), fontsize=room_lgdsize)

# plt.xticks(np.arange(10, 41, 10))
# plt.yticks(np.arange(-1,.5,.5))
# plt.show()

In [ ]:
# c_label = ["c_N","c_NH3","c_NO3"]
# c_sd_label = ["c_N_SD", "c_NH3_SD", "c_NO3_SD"]
# conc_ylabel = "Concentration, $c\ [\si{\milli\mole\per\liter}]$"

# zea_cn_df = pd.read_csv(data_path + 'zea_cN.csv')
# zea_cn_df = zea_cn_df[zea_cn_df[time_label] <= harvest_cutoff]

# zea_cn_hg = df_n_selection(zea_cn_df, nitrogen_label, 1).grouped
# zea_cn_md = df_n_selection(zea_cn_df, nitrogen_label, 0).grouped
# zea_cn_lw = df_n_selection(zea_cn_df, nitrogen_label, -1).grouped

# #zea_cn_hg[time_label] += point_offset
# #zea_cn_lw[time_label] -= point_offset

# zea_cn_total_hg = zea_cn_hg[[time_label, c_label[0], c_sd_label[0]]].copy()
# zea_cn_total_md = zea_cn_md[[time_label, c_label[0], c_sd_label[0]]].copy()
# zea_cn_total_lw = zea_cn_lw[[time_label, c_label[0], c_sd_label[0]]].copy()

# zea_cn_NH3_hg = zea_cn_hg[[time_label, c_label[1], c_sd_label[1]]].copy()
# zea_cn_NH3_hg[time_label] += point_offset
# zea_cn_NH3_md = zea_cn_md[[time_label, c_label[1], c_sd_label[1]]].copy()
# zea_cn_NH3_md[time_label] += point_offset
# zea_cn_NH3_lw = zea_cn_lw[[time_label, c_label[1], c_sd_label[1]]].copy()
# zea_cn_NH3_lw[time_label] += point_offset

# zea_cn_NO3_hg = zea_cn_hg[[time_label, c_label[2], c_sd_label[2]]].copy()
# zea_cn_NO3_hg[time_label] -= point_offset
# zea_cn_NO3_md = zea_cn_md[[time_label, c_label[2], c_sd_label[2]]].copy()
# zea_cn_NO3_md[time_label] -= point_offset
# zea_cn_NO3_lw = zea_cn_lw[[time_label, c_label[2], c_sd_label[2]]].copy()
# zea_cn_NO3_lw[time_label] -= point_offset

# nss_c_fig, nss_c_axes = plt.subplots(nrows=3, ncols=1, sharex=True, sharey=True)
# nss_c_fig.set_figheight(15)
# nss_c_fig.set_figwidth(15)

# nss_c_axes[0].errorbar(zea_cn_total_hg[time_label], zea_cn_total_hg[c_label[0]], yerr=zea_cn_total_hg[c_sd_label[0]], label=ln_label[0], color=totalN_color)
# nss_c_axes[0].errorbar(zea_cn_NH3_hg[time_label], zea_cn_NH3_hg[c_label[1]], yerr=zea_cn_NH3_hg[c_sd_label[1]], label=ln_label[1], color=ammonia_color, ls='--')
# nss_c_axes[0].errorbar(zea_cn_NO3_hg[time_label], zea_cn_NO3_hg[c_label[2]], yerr=zea_cn_NO3_hg[c_sd_label[2]], label=ln_label[2], color=nitrate_color, ls='--')
# nss_c_axes[0].set_title("Excess Nitrogen Treatment")
# nss_c_axes[1].errorbar(zea_cn_total_md[time_label], zea_cn_total_md[c_label[0]], yerr=zea_cn_total_md[c_sd_label[0]], label=ln_label[0], color=totalN_color)
# nss_c_axes[1].errorbar(zea_cn_NH3_md[time_label], zea_cn_NH3_md[c_label[1]], yerr=zea_cn_NH3_md[c_sd_label[1]], label=ln_label[1], color=ammonia_color, ls='--')
# nss_c_axes[1].errorbar(zea_cn_NO3_md[time_label], zea_cn_NO3_md[c_label[2]], yerr=zea_cn_NO3_md[c_sd_label[2]], label=ln_label[2], color=nitrate_color, ls='--')
# nss_c_axes[1].set_title("Normal Nitrogen Treatment")
# nss_c_axes[2].errorbar(zea_cn_total_lw[time_label], zea_cn_total_lw[c_label[0]], yerr=zea_cn_total_lw[c_sd_label[0]], label=ln_label[0], color=totalN_color)
# nss_c_axes[2].errorbar(zea_cn_NH3_lw[time_label], zea_cn_NH3_lw[c_label[1]], yerr=zea_cn_NH3_lw[c_sd_label[1]], label=ln_label[1], color=ammonia_color, ls='--')
# nss_c_axes[2].errorbar(zea_cn_NO3_lw[time_label], zea_cn_NO3_lw[c_label[2]], yerr=zea_cn_NO3_lw[c_sd_label[2]], label=ln_label[2], color=nitrate_color, ls='--')
# nss_c_axes[2].set_title("Deficient Nitrogen Treatment")

# nss_c_fig.supxlabel(N_time_xlabel)
# nss_c_fig.supylabel(conc_ylabel)

# nss_c_fig.subplots_adjust(bottom=0.1)  # we need these hax so the legend doesn't go on top of x label
# nss_c_fig.legend(labels=["Total N","NH$_3$","NO$_3^-$"], ncol=3, loc="lower center",  bbox_to_anchor=(0.5, -0.05))
# plt.show()

## Relative uptake rate
Not used; this is from an example calculation

In [ ]:
# zea_RUR_df = pd.read_csv(data_path + 'zea_RUR.csv')
# zea_RUR_df = zea_RUR_df[zea_RUR_df[time_label] <= 45]

# zea_RUR_hg = df_n_selection(zea_RUR_df, nitrogen_label, 1).grouped
# zea_RUR_md = df_n_selection(zea_RUR_df, nitrogen_label, 0).grouped
# zea_RUR_lw = df_n_selection(zea_RUR_df, nitrogen_label, -1).grouped

# zea_RUR_hg[time_label] = zea_RUR_hg[time_label] + emerge_offset + point_offset
# zea_RUR_md[time_label] += emerge_offset
# zea_RUR_lw[time_label] = zea_RUR_lw[time_label] + emerge_offset - point_offset

# RUR_hg_color = "orange"
# RUR_md_color = "gold"
# RUR_lw_color = "burlywood"
# RUR_lnwt = 5

# fig_RUR, ax_RUR = plt.subplots(1, figsize=(10, 10))
# zea_RUR_hg.plot(x=time_label, y="RUR", yerr="SD_RUR", label=hg_label, color=RUR_hg_color, ls='-.', lw=RUR_lnwt, ax=ax_RUR)
# zea_RUR_md.plot(x=time_label, y="RUR", yerr="SD_RUR", label=md_label, color=RUR_md_color, ls='-.', lw=RUR_lnwt, ax=ax_RUR)
# zea_RUR_lw.plot(x=time_label, y="RUR", yerr="SD_RUR", label=lw_label, color=RUR_lw_color, ls='-.', lw=RUR_lnwt, ax=ax_RUR)

# ax_RUR.set_xlabel(time_xlabel)
# # ax_RUR.xaxis.set_minor_locator(ticker.MultipleLocator(1))
# ax_RUR.set_ylabel("Relative Uptake Rate, RUR $[\si{\gram\of{N}\per\day\per\gram\of{DW}}]$")
# ax_RUR.set_box_aspect(1)

# plt.ticklabel_format(axis='y', style='sci', scilimits=(-3,-3))
# plt.legend(ncol=2, loc="lower center", bbox_to_anchor=(0.5, -0.35))
# plt.show()

# fig_RUR.savefig("./NP-images/RUR.png", dpi=300, bbox_inches='tight', transparent=True)

## RGR vs. N%
Unused. Maybe there is a correlation!

In [ ]:
# zea_pctN_df = pd.read_csv(data_path + 'zea_pctN.csv')
# zea_pctN_df = zea_pctN_df[zea_pctN_df[time_label] <= harvest_cutoff_zea]

# zea_pctN_hg = df_n_selection(zea_pctN_df, nitrogen_label, 1).grouped
# zea_pctN_md = df_n_selection(zea_pctN_df, nitrogen_label, 0).grouped
# zea_pctN_lw = df_n_selection(zea_pctN_df, nitrogen_label, -1).grouped

# zea_pctN_hg[time_label] += emerge_offset
# zea_pctN_md[time_label] += emerge_offset
# zea_pctN_lw[time_label] += emerge_offset

# zea_rgr_df = pd.read_csv(data_path + 'zea_RGR.csv')
# zea_rgr_df = zea_rgr_df[zea_rgr_df[time_label] <= harvest_cutoff_zea]

# zea_rgr_hg = df_n_selection(zea_rgr_df, nitrogen_label, 1).grouped
# zea_rgr_md = df_n_selection(zea_rgr_df, nitrogen_label, 0).grouped
# zea_rgr_lw = df_n_selection(zea_rgr_df, nitrogen_label, -1).grouped

# zea_rgr_hg[time_label] += emerge_offset
# zea_rgr_md[time_label] += emerge_offset
# zea_rgr_lw[time_label] += emerge_offset

# RGRvN_hg_color = "DarkMagenta"
# RGRvN_md_color = "HotPink"
# RGRvN_lw_color = "Plum"

# RGR_pctN_fig, RGR_pctN_axes = plt.subplots(1, figsize=(10, 10))

# RGR_pctN_axes.set_xlabel("Plant Nitrogen Content [\% w/w]")
# RGR_pctN_axes.set_ylabel("Relative Growth Rate, RGR $[\si{\gram\of{DW}\per\day\per\gram\of{DW}}]$")
# RGR_pctN_axes.set_box_aspect(1)

# z_hg = np.polyfit(zea_pctN_hg['pctN'], zea_rgr_hg['RGR'], 1)
# p_hg = np.poly1d(z_hg)

# z_md = np.polyfit(zea_pctN_md['pctN'], zea_rgr_md['RGR'], 1)
# p_md = np.poly1d(z_md)

# z_lw = np.polyfit(zea_pctN_lw['pctN'], zea_rgr_lw['RGR'], 1)
# p_lw = np.poly1d(z_lw)

# # import scipy.stats
# # slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(zea_pctN_lw['pctN'], zea_rgr_lw['RGR'])
# # print(slope)
# # print(r_value)

# trend_rg = np.arange(0.1, 0.7, 0.1)

# RGR_pctN_lnwt = 5

# # print("y=%.6fx+(%.6f)"%(z[0],z[1]))

# RGR_pctN_axes.errorbar(zea_pctN_hg['pctN'], zea_rgr_hg['RGR'], xerr=zea_pctN_hg['std_pctN'], yerr=zea_rgr_hg['std_RGR'], color=RGRvN_hg_color, label=hg_label, fmt='.', lw=4)
# RGR_pctN_axes.errorbar(zea_pctN_md['pctN'], zea_rgr_md['RGR'], xerr=zea_pctN_md['std_pctN'], yerr=zea_rgr_md['std_RGR'], color=RGRvN_md_color, label=md_label, fmt='.', lw=4)
# RGR_pctN_axes.errorbar(zea_pctN_lw['pctN'], zea_rgr_lw['RGR'], xerr=zea_pctN_lw['std_pctN'], yerr=zea_rgr_lw['std_RGR'], color=RGRvN_lw_color, label=lw_label, fmt='.', lw=4)

# zea_pctN_md = zea_pctN_md.sort_values(by='pctN') # fix backtracking
# RGR_pctN_axes.plot(zea_pctN_hg['pctN'],p_hg(zea_pctN_hg['pctN']), color=RGRvN_hg_color, ls="--", lw=RGR_pctN_lnwt)
# RGR_pctN_axes.plot(zea_pctN_md['pctN'],p_md(zea_pctN_md['pctN']), color=RGRvN_md_color, ls="--", lw=RGR_pctN_lnwt)
# RGR_pctN_axes.plot(zea_pctN_lw['pctN'],p_lw(zea_pctN_lw['pctN']), color=RGRvN_lw_color, ls="--", lw=RGR_pctN_lnwt)

# RGR_pctN_axes.xaxis.set_minor_locator(ticker.MultipleLocator(0.05))

# #order=[2,1,0]
# handles, labels = plt.gca().get_legend_handles_labels()
# #plt.legend([handles[i] for i in order],[labels[i] for i in order], ncol=2, loc="lower center",  bbox_to_anchor=(0.5, -0.35))
# plt.legend(ncol=2, loc="lower center",  bbox_to_anchor=(0.5, -0.35))
# plt.xticks(np.arange(0.1, 0.7, 0.1))
# plt.yticks(np.arange(-0.1, 0.5, 0.1))
# plt.show()

# RGR_pctN_fig.savefig("./NP-images/RGR-vs-pctN.png", dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# print("y=%.6fx+(%.6f)"%(z_hg[0],z_hg[1]))
# print("y=%.6fx+(%.6f)"%(z_md[0],z_md[1]))
# print("y=%.6fx+(%.6f)"%(z_lw[0],z_lw[1]))

## mu_N (RNAR)
Unused. From an example calculation.

In [ ]:
# '''
# Figure: rnar

# '''
# zea_rnar_df = pd.read_csv(data_path + 'zea_RNAR.csv')
# zea_rnar_df = zea_rnar_df[zea_rnar_df[time_label] <= harvest_cutoff_zea]
# zea_rnar_hg = df_n_selection(zea_rnar_df, nitrogen_label, 1).grouped
# zea_rnar_md = df_n_selection(zea_rnar_df, nitrogen_label, 0).grouped
# zea_rnar_lw = df_n_selection(zea_rnar_df, nitrogen_label, -1).grouped

# zea_rnar_hg[time_label] = zea_rnar_hg[time_label] + point_offset + emerge_offset
# zea_rnar_md[time_label] += emerge_offset
# zea_rnar_lw[time_label] = zea_rnar_lw[time_label] - point_offset + emerge_offset

# z_hg = np.polyfit(zea_rnar_hg[time_label], zea_rnar_hg['RNAR'], 1)
# p_hg = np.poly1d(z_hg)

# z_md = np.polyfit(zea_rnar_md[time_label], zea_rnar_md['RNAR'], 1)
# p_md = np.poly1d(z_md)

# z_lw = np.polyfit(zea_rnar_lw[time_label], zea_rnar_lw['RNAR'], 1)
# p_lw = np.poly1d(z_lw)

# rnar_hg_color = "orange"
# rnar_md_color = "gold"
# rnar_lw_color = "burlywood"

# # Figure
# fig_rnar, ax_rnar = plt.subplots(1, figsize=(10, 10))
# ax_rnar.set_box_aspect(1)

# # Draw plots
# rnar_lnwt = 5
# zea_rnar_hg.plot(x=time_label, y="RNAR", yerr="std_RNAR", label=hg_label, color=rnar_hg_color, ls='-', lw=rnar_lnwt, ax=ax_rnar)
# zea_rnar_md.plot(x=time_label, y="RNAR", yerr="std_RNAR", label=md_label, color=rnar_md_color, ls='-', lw=rnar_lnwt, ax=ax_rnar)
# zea_rnar_lw.plot(x=time_label, y="RNAR", yerr="std_RNAR", label=lw_label, color=rnar_lw_color, ls='-', lw=rnar_lnwt, ax=ax_rnar)

# ax_rnar.plot(zea_rnar_hg[time_label], p_hg(zea_rnar_hg[time_label]), color=rnar_hg_color, ls="--", lw=rnar_lnwt)
# ax_rnar.plot(zea_rnar_md[time_label], p_md(zea_rnar_md[time_label]), color=rnar_md_color, ls="--", lw=rnar_lnwt)
# ax_rnar.plot(zea_rnar_lw[time_label], p_lw(zea_rnar_lw[time_label]), color=rnar_lw_color, ls="--", lw=rnar_lnwt)

# # Set axes
# ax_rnar.set_xlabel(time_xlabel)
# plt.xticks(np.arange(20, harvest_cutoff + 1, 5))
# ax_rnar.xaxis.set_minor_locator(ticker.MultipleLocator(1))

# ax_rnar.set_ylabel("Relative N Accum. Rate, $\mu_\mathrm{N}\ [\si{\per\day}]$")
# plt.yticks(np.arange(-0.1, 0.5, 0.1))
# ax_rnar.set_xlim(19.5,40.5)
# ax_rnar.xaxis.set_major_locator(ticker.MultipleLocator(5))
# ax_rnar.xaxis.set_minor_locator(ticker.MultipleLocator(1))

# # Draw t_M box
# ax_rnar.axvspan(30, 35, color="#a5a5a5", alpha=0.25) # label="Harvesting Range"

# # Legend
# plt.legend(ncol=2, loc="lower center",  bbox_to_anchor=(0.5, -0.35))

# plt.show()

# fig_rnar.savefig("./NP-images/RNAR.png", dpi=300, bbox_inches='tight', transparent=True)

## eta_N
Not used; this is from an example calculation

In [ ]:
# '''
# Figure: NUE

# '''
# zea_etaN_df = pd.read_csv(data_path + 'zea_NUE.csv')
# zea_etaN_df = zea_etaN_df[zea_etaN_df[time_label] <= harvest_cutoff_zea]
# zea_etaN_hg = df_n_selection(zea_etaN_df, nitrogen_label, 1).grouped
# zea_etaN_md = df_n_selection(zea_etaN_df, nitrogen_label, 0).grouped
# zea_etaN_lw = df_n_selection(zea_etaN_df, nitrogen_label, -1).grouped

# zea_etaN_hg[time_label] = zea_etaN_hg[time_label] + point_offset + emerge_offset
# zea_etaN_md[time_label] += emerge_offset
# zea_etaN_lw[time_label] = zea_etaN_lw[time_label] - point_offset + emerge_offset

# z_hg = np.polyfit(zea_etaN_hg[time_label], zea_etaN_hg['NUE'], 1)
# p_hg = np.poly1d(z_hg)

# z_md = np.polyfit(zea_etaN_md[time_label], zea_etaN_md['NUE'], 1)
# p_md = np.poly1d(z_md)

# z_lw = np.polyfit(zea_etaN_lw[time_label], zea_etaN_lw['NUE'], 1)
# p_lw = np.poly1d(z_lw)

# etaN_hg_color = "DarkMagenta"
# etaN_md_color = "Hotpink"
# etaN_lw_color = "Plum"
# etaN_lnwt = 5
# etaN_args = {'ls':'-', 'lw':etaN_lnwt}

# # Figure
# fig_etaN, ax_etaN = plt.subplots(1, figsize=(10, 10))
# ax_etaN.set_box_aspect(1)

# # Draw plots
# zea_etaN_hg.plot(x=time_label, y="NUE", yerr="std_NUE", label=hg_label, color=etaN_hg_color, ax=ax_etaN, **etaN_args)
# zea_etaN_md.plot(x=time_label, y="NUE", yerr="std_NUE", label=md_label, color=etaN_md_color, ax=ax_etaN, **etaN_args)
# zea_etaN_lw.plot(x=time_label, y="NUE", yerr="std_NUE", label=lw_label, color=etaN_lw_color, ax=ax_etaN, **etaN_args)

# ax_etaN.plot(zea_etaN_hg[time_label], p_hg(zea_etaN_hg[time_label]), color=etaN_hg_color, ls="--", lw=etaN_lnwt)
# ax_etaN.plot(zea_etaN_md[time_label], p_md(zea_etaN_md[time_label]), color=etaN_md_color, ls="--", lw=etaN_lnwt)
# ax_etaN.plot(zea_etaN_lw[time_label], p_lw(zea_etaN_lw[time_label]), color=etaN_lw_color, ls="--", lw=etaN_lnwt)

# # Set axes
# ax_etaN.set_xlabel(time_xlabel)
# plt.xticks(np.arange(20, harvest_cutoff + 1, 5))
# ax_etaN.set_xlim(19.5,40.5)
# ax_etaN.xaxis.set_major_locator(ticker.MultipleLocator(5))
# ax_etaN.xaxis.set_minor_locator(ticker.MultipleLocator(1))

# ax_etaN.set_ylabel("N Use Efficiency, $\eta_\mathrm{N}\ [\si{\gram\of{DW}\per\gram\of{N}}]$")
# # plt.yticks(np.arange(-0.1, 0.5, 0.1))
# ax_etaN.yaxis.set_minor_locator(ticker.MultipleLocator(5))

# # Draw t_M box
# ax_etaN.axvspan(30, 35, color="#a5a5a5", alpha=0.25) # label="Harvesting Range"

# # Legend
# plt.legend(ncol=2, loc="lower center",  bbox_to_anchor=(0.5, -0.35))

# plt.show()

# fig_etaN.savefig("./NP-images/etaN.png", dpi=300, bbox_inches='tight', transparent=True)

## Plot cleanroom parameters

In [ ]:
filename = "./zea-data/zea_room.csv"
df = pd.read_csv(filename)

room_x = df['dAS']
room_temp = df['T']
room_hr = df['HR']
room_cCO2 = df['cCO2']

fig_room, ax_room = plt.subplots(nrows=1, ncols=3, sharex=True)

fig_room.set_figheight(5)
fig_room.set_figwidth(16)

# room_lgdsize = lgd_font_size
room_axissize = 18

room_temp_clr = 'k'
room_hr_clr = 'k'
room_co2_clr = 'k'

for ax in ax_room:
    ax.set_adjustable('box', share=True)
    ax.tick_params(axis='both', labelsize=room_axissize)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(10))
    ax.set_xlim(-1,50)
    # ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.25))

ax_room[0].plot(room_x, room_temp, color=room_temp_clr, ls='', marker='o')
ax_room[0].hlines(y=24, xmin=0, xmax=50, linewidth=2, ls='--', color=room_temp_clr)
ax_room[0].hlines(y=19, xmin=0, xmax=50, linewidth=2, ls='--', color=room_temp_clr)
ax_room[0].set_ylabel("Air Temperature, $T\ [\si{\degreeCelsius}]$", fontsize=room_axissize)
ax_room[0].yaxis.set_major_locator(ticker.MultipleLocator(2))
ax_room[0].yaxis.set_minor_locator(ticker.MultipleLocator(1))
ax_room[0].set_ylim(18,25)

ax_room[1].plot(room_x, room_hr, color=room_hr_clr, ls='', marker='v')
ax_room[1].hlines(y=40, xmin=0, xmax=50, linewidth=2, ls='--', color=room_hr_clr)
ax_room[1].hlines(y=60, xmin=0, xmax=50, linewidth=2, ls='--', color=room_hr_clr)
ax_room[1].set_ylabel("Relative Humidity, $h_\mathrm{R}\ [\si{\percent}]$", fontsize=room_axissize)
ax_room[1].yaxis.set_major_locator(ticker.MultipleLocator(10))
ax_room[1].yaxis.set_minor_locator(ticker.MultipleLocator(2))
ax_room[1].set_ylim(35,65)

ax_room[2].plot(room_x, room_cCO2, color=room_co2_clr, ls='', marker='^')
ax_room[2].set_ylabel("Conc. CO$_2$, $c_\mathrm{CO2}$ [ppm]", fontsize=room_axissize)
ax_room[2].hlines(y=400, xmin=0, xmax=50, linewidth=2, ls='--', color=room_co2_clr)
ax_room[2].hlines(y=650, xmin=0, xmax=50, linewidth=2, ls='--', color=room_co2_clr)
ax_room[2].yaxis.set_major_locator(ticker.MultipleLocator(100))
ax_room[2].yaxis.set_minor_locator(ticker.MultipleLocator(50))
ax_room[2].set_ylim(350,700)

fig_room.supxlabel(N_time_xlabel, fontsize=room_axissize, y=0.2)

fig_room.tight_layout()
plt.show()

fig_room.savefig("./NP-images/room.png", dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# https://stackoverflow.com/questions/23144784/plotting-error-bars-on-grouped-bars-in-pandas
from matplotlib import cm

# Let's just make a dataframe manually
ppfd_df = pd.DataFrame([[229.70625,232.525,234.21875,215.85625,243.54375,14.6445197372032,13.823144842377,13.7313858853844,14.0641847138989,14.5454675070965],
                   [234.45625,245.775,245.1125,229.23125,253.3,19.9486664132384,16.8596362159251,15.2563811785976,20.5420452973895,16.9325327894756],
                  [223.05,218.2875,228.5375,218.675,248.69375,15.8965405041474,19.2030509728706,14.414847669446,15.0606108773848,16.3408881337582]], 
                  columns=["Week 0","Week 1","Week 2","Week 3","Week 7","SD0","SD1","SD2","SD3","SD7"], index=["Deficient","Normal","Excess"])
yerr = ppfd_df[["SD0", "SD1", "SD2", "SD3", "SD7"]].to_numpy().T

ppfd_args = {'lines.markeredgewidth':1, 'errorbar.capsize':3, 'axes.linewidth':1, 'xtick.major.size':10, 'xtick.major.width':1, 'xtick.minor.size':1, 'xtick.minor.width':1, 
             'ytick.major.size':5, 'ytick.major.width':1.5, 'ytick.minor.size':2.5, 'ytick.minor.width':1.5, 'lines.linewidth':1}

with plt.rc_context(ppfd_args):  # override rcParams
    plt_ppfd = ppfd_df[["Week 0","Week 1","Week 2","Week 3","Week 7"]].plot(kind='bar', yerr=yerr, alpha=1, error_kw=dict(ecolor='k'), colormap=cm.summer, fontsize=room_axissize, figsize=(7,4), align='center', width=0.75)
    plt.hlines(y=250, xmin=-1000, xmax=100, linewidth=1, ls='--', color='k')
    plt.hlines(y=200, xmin=-1000, xmax=100, linewidth=1, ls='--', color='k')

plt_ppfd.yaxis.set_major_locator(ticker.MultipleLocator(50))
plt_ppfd.yaxis.set_minor_locator(ticker.MultipleLocator(25))
plt_ppfd.set_ylim(0,275)
plt_ppfd.tick_params(bottom=False)
plt_ppfd.legend(bbox_to_anchor=(1.0, 0.75),fontsize=room_axissize-4)
plt.xticks(rotation=0)  # why does this only work with plt...?
plt.xlabel("\nNitrogen Condition", fontsize=room_axissize)
# plt.ylabel("Photosynthetic Photon Flux, $\Phi_\gamma [\si{\micro\mol\of{$\gamma$}\per\second\per\meter\squared}]$", fontsize=room_axissize)
plt.ylabel(r'$\Phi_\gamma\ [\si{\micro\mole\of{\gamma}\per\meter\squared\per\second}]$', fontsize=room_axissize)

plt_ppfd.get_figure().savefig("./NP-images/ppfd.png", dpi=300, bbox_inches='tight', transparent=True)

plt.show()

## Plot N removed

In [ ]:
# We use "dAE" as the time column since we have numbers and strings in it and offsetting is annoying
zea_Nrm_df = pd.read_csv(data_path + 'zea_N-removed.csv')

zea_Nrm_df.rename(columns={"lw_mN": "Deficient", "md_mN": "Normal", "hg_mN": "Excess", "all_mN": "All"}, inplace=True)

# The daily total doesn't really add value
zea_Nrm_df = zea_Nrm_df.drop(labels="All", axis=1)

# Figure
fig_Nrm, ax_Nrm = plt.subplots(1, figsize=(10, 7))

# Draw plots
zea_Nrm_df.plot.bar(x="dAE", ax=ax_Nrm, color=[lw_mB_color, md_mB_color, hg_mB_color], rot=0, width=0.85, logy=False, fontsize=28)

# Set axes
ax_Nrm.set_xlabel(time_xlabel)

ax_Nrm.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
ax_Nrm.yaxis.set_major_locator(ticker.MultipleLocator(1.0))
ax_Nrm.yaxis.set_minor_locator(ticker.MultipleLocator(0.5))
ax_Nrm.set_ylim(0,6)
# ax_Nrm.tick_params(axis='both', which='major', labelsize=14)

ax_Nrm.set_ylabel("Nitrogen in Harvest $[\si{\gram\of{N}}]$", fontsize=plt_font_size)

# Legend
ax_Nrm.legend(ncol=3, fontsize=lgd_font_size, loc='upper left', columnspacing=0.75, handletextpad=0.2)

plt.show()

fig_Nrm.savefig("./NP-images/N-removed.png", dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
# We use "dAE" as the time column since we have numbers and strings in it and offsetting is annoying
zea_Nrm_df = pd.read_csv(data_path + 'zea_N-removed.csv')

zea_Nrm_df.rename(columns={"lw_mN": "Deficient", "md_mN": "Normal", "hg_mN": "Excess", "all_mN": "All"}, inplace=True)
zea_Nrm_df['dAE'] = zea_Nrm_df['dAE'].astype("category")
# The daily total doesn't really add value, so drop it
zea_Nrm_df = zea_Nrm_df.drop(labels="All", axis=1)

# read in avg. mB. Rename annoying headers. 
zea_mB_avg_hg = pd.read_csv(data_path + "zea_biomass-dw-area_excess.csv")
zea_mB_avg_hg.rename(columns={"mB mean [gDW/m^2]": "mB_avg_hg", "stdev": "mB_avg_SD_hg"}, inplace=True)
zea_mB_avg_md = pd.read_csv(data_path + "zea_biomass-dw-area_normal.csv")
zea_mB_avg_md.rename(columns={"mB mean [gDW/m^2]": "mB_avg_md", "stdev": "mB_avg_SD_md"}, inplace=True)
zea_mB_avg_lw = pd.read_csv(data_path + "zea_biomass-dw-area_deficient.csv")
zea_mB_avg_lw.rename(columns={"mB mean [gDW/m^2]": "mB_avg_lw", "stdev": "mB_avg_SD_lw"}, inplace=True)

# Figure
fig_Nrm, ax_Nrm = plt.subplots(1, figsize=(10, 10))
ax_Nrm2 = ax_Nrm.twinx()
# ax_Nrm3 = ax_Nrm.twinx()

conditions = ["hg", "md", "lw"]
pctN_d11_est = {"hg":0.595155, "md":0.56632, "lw":0.62266}  # H, M, L

def proc_Nrm_Npct_df(df, cond):
    '''
    Takes original N_pct df and whatever condition it described, then
    * Adds day 11
    * makes a copy of condition's pct_N
    * removes offset from day column and makes it a string
    * gives columns names unique to condition
    I should have just made a new csv.
    '''
    d_11_line = pd.DataFrame({'day': 11, 'pct_N': pctN_d11_est[cond], 'SD_pct_N': 0},index=[0])
    
    df_copy = df[[time_label, "pct_N", "SD_pct_N"]].copy()
    df_copy = pd.concat([d_11_line, df_copy]).reset_index(drop = True)
    df_copy["day"] = df_copy["day"].round().astype(int).astype("string")
    df_copy.rename(columns={"day": "dAE", "pct_N": "pct_N"+"_"+cond, "SD_pct_N": "SD_pct_N"+"_"+cond}, inplace=True)
    return df_copy

# We refer to mN because pctN is in the same dataframe
zea_mN_hg_Nrm = proc_Nrm_Npct_df(zea_mN_hg,conditions[0])
zea_mN_md_Nrm = proc_Nrm_Npct_df(zea_mN_md,conditions[1])
zea_mN_lw_Nrm = proc_Nrm_Npct_df(zea_mN_lw,conditions[2])

# zea_mN_hg_Nrm.plot(x="dAE", y="pct_N", yerr="SD_pct_N", label=hg_label, color=mN_hg_color, **pctN_plt_param, ax=ax_Nrm2)
# zea_mN_md.plot(x=time_label, y="pct_N", yerr="SD_pct_N", label=md_label, color=mN_md_color, **pctN_plt_param, ax=ax_pctN)
# zea_mN_lw.plot(x=time_label, y="pct_N", yerr="SD_pct_N", label=lw_label, color=mN_lw_color, **pctN_plt_param, ax=ax_pctN)

zea_Nrm_df = zea_Nrm_df.merge(zea_mN_hg_Nrm, how='outer')
zea_Nrm_df = zea_Nrm_df.merge(zea_mN_md_Nrm, how='outer')
zea_Nrm_df = zea_Nrm_df.merge(zea_mN_lw_Nrm, how='outer')

zea_mB_avg_hg["dAE"] = zea_mB_avg_hg["dAE"].round().astype(int).astype("string")
zea_Nrm_df = zea_Nrm_df.merge(zea_mB_avg_hg, how='outer')
zea_mB_avg_md["dAE"] = zea_mB_avg_md["dAE"].round().astype(int).astype("string")
zea_Nrm_df = zea_Nrm_df.merge(zea_mB_avg_md, how='outer')
zea_mB_avg_lw["dAE"] = zea_mB_avg_lw["dAE"].round().astype(int).astype("string")
zea_Nrm_df = zea_Nrm_df.merge(zea_mB_avg_lw, how='outer')

zea_Nrm_df[["dAE","Deficient","Normal","Excess"]].plot.bar(x="dAE", ax=ax_Nrm, color=[lw_mB_color, md_mB_color, hg_mB_color], rot=0, width=0.85, logy=False, fontsize=28)
zea_Nrm_df.plot(x="dAE", y="pct_N_hg", yerr="SD_pct_N_hg", label=hg_label, color=mN_hg_color, **pctN_plt_param, ax=ax_Nrm2)
zea_Nrm_df.plot(x="dAE", y="pct_N_md", yerr="SD_pct_N_md", label=md_label, color=mN_md_color, **pctN_plt_param, ax=ax_Nrm2)
zea_Nrm_df.plot(x="dAE", y="pct_N_lw", yerr="SD_pct_N_lw", label=lw_label, color=mN_lw_color, **pctN_plt_param, ax=ax_Nrm2)
# zea_Nrm_df.plot(x="dAE", y="mB_avg_hg", yerr="mB_avg_SD_hg", label=hg_label, color=mN_hg_color, ax=ax_Nrm3, ls=None)
# zea_Nrm_df.plot(x="dAE", y="mB_avg_md", yerr="mB_avg_SD_md", label=md_label, color=mN_md_color, ax=ax_Nrm3, ls="")
# zea_Nrm_df.plot(x="dAE", y="mB_avg_lw", yerr="mB_avg_SD_lw", label=md_label, color=mN_lw_color, ax=ax_Nrm3, ls="")

# Set axes
ax_Nrm.set_xlabel(time_xlabel)

ax_Nrm.yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
ax_Nrm.yaxis.set_major_locator(ticker.MultipleLocator(1.0))
ax_Nrm.yaxis.set_minor_locator(ticker.MultipleLocator(0.5))
# ax_Nrm.set_ylim(0,6)
# ax_Nrm.tick_params(axis='both', which='major', labelsize=14)

ax_Nrm2.set_ylim(0,0.8)
ax_Nrm2.spines.right.set_visible(True)

ax_Nrm2.set_ylabel("Plant Nitrogen Content [\% w/w]")
ax_Nrm2.yaxis.set_minor_locator(ticker.MultipleLocator(0.1))

# ax_Nrm3.spines['right'].set_position(('axes', 1.2))
# Draw t_M box and label
# ax_Nrm.axvspan(30, 35, color="#a5a5a5", alpha=0.25) # label="Harvesting Range"
# ax_Nrm.text(0, ax_Nrm2.get_ylim()[1]*0.9, r"$t_\textrm{M}$", fontsize=lgd_font_size+2, ha='center')

ax_Nrm.set_ylabel("Nitrogen in Harvest $[\si{\gram\of{N}}]$", fontsize=plt_font_size)

# Legend
ax_Nrm.legend(ncol=3, fontsize=lgd_font_size, loc='upper left', columnspacing=0.75, handletextpad=0.2)
ax_Nrm2.get_legend().remove()

plt.show()

# fig_Nrm.savefig("./NP-images/N-removed.png", dpi=300, bbox_inches='tight', transparent=True)

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(12, 11), sharex=True, height_ratios=[1, 1, 2])

zea_Nrm_df[["dAE","Deficient","Normal","Excess"]].plot.bar(x="dAE", ax=axs[2], color=[lw_mB_color, md_mB_color, hg_mB_color], rot=0, width=0.85, logy=False, fontsize=28)
zea_Nrm_df.plot(x="dAE", y="pct_N_hg", label=hg_label, color=mN_hg_color, **pctN_plt_param, ax=axs[1])
zea_Nrm_df.plot(x="dAE", y="pct_N_md", label=md_label, color=mN_md_color, **pctN_plt_param, ax=axs[1])
zea_Nrm_df.plot(x="dAE", y="pct_N_lw", label=lw_label, color=mN_lw_color, **pctN_plt_param, ax=axs[1])
zea_Nrm_df.plot(x="dAE", y="mB_avg_hg", label=hg_label, color=mN_hg_color, ax=axs[0], **pctN_plt_param)
zea_Nrm_df.plot(x="dAE", y="mB_avg_md", label=md_label, color=mN_md_color, ax=axs[0], **pctN_plt_param)
zea_Nrm_df.plot(x="dAE", y="mB_avg_lw", label=lw_label, color=mN_lw_color, ax=axs[0], **pctN_plt_param)

axs[2].set_xlabel(time_xlabel)

axs[2].set_ylim(0,6)
axs[2].yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
axs[2].yaxis.set_major_locator(ticker.MultipleLocator(2.0))
axs[2].yaxis.set_minor_locator(ticker.MultipleLocator(1.0))
# axs[2].set_title("Nitrogen in Harvest", loc='left', fontsize=plt_font_size)
axs[2].set_ylabel("$[\si{\gram\of{N}}]$", fontsize=plt_font_size)

axs[0].set_ylim(-12,120)
axs[0].yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
axs[0].yaxis.set_major_locator(ticker.MultipleLocator(40))
axs[0].yaxis.set_minor_locator(ticker.MultipleLocator(20))
# axs[0].set_title("Biomass", loc='left', fontsize=plt_font_size)
axs[0].set_ylabel("[$\si{\gram\of{DW}\per\meter\squared}$]", fontsize=plt_font_size)

axs[1].set_ylim(-0.075,0.75)
axs[1].yaxis.set_major_formatter(ticker.FormatStrFormatter('%.1f'))
axs[1].yaxis.set_major_locator(ticker.MultipleLocator(0.25))
axs[1].yaxis.set_minor_locator(ticker.MultipleLocator(0.125))
# axs[1].set_title("Nitrogen Content", loc='left', fontsize=plt_font_size)
axs[1].set_ylabel("[\% w/w]", fontsize=plt_font_size)

for a in axs: a.get_legend().remove()
fig.align_ylabels(axs[:])

plt.show()

In [ ]:
zea_Nrm_df